# Fantasy Football Predictions

This begins the project for predicting fantasy football scores by player for the upcoming 2023 season.

## Notebook 1: Data Gathering, Cleaning, and Merging

First, data needs to be collected and organized in a way that will be helpful for future exploratory data analysis and modeling.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# importing API to connect to NFL data sources like nflfastR, nfldata, dynastyprocess, and Draft Scout.
# via cooperdff
import nfl_data_py as nfl 

import requests
from bs4 import BeautifulSoup

# Data ingestion

In [2]:
# fantasy football datasets; results from each season

# ff22 = pd.read_csv('C:/Users/steph/OneDrive/Documents/Data Projects/Fantasy/2022_ff_stats.csv')
# can't find similar ff22 data,so going to scrape from pro football reference

ff21 = pd.read_csv('https://raw.githubusercontent.com/fantasydatapros/data/master/yearly/2021.csv', index_col = 0)

ff20 = pd.read_csv('https://raw.githubusercontent.com/fantasydatapros/data/master/yearly/2020.csv')

ff19 = pd.read_csv('https://raw.githubusercontent.com/fantasydatapros/data/master/yearly/2019.csv', index_col = 0)

# all nfl team names and abbreviations

nflteams = pd.read_csv('https://gist.githubusercontent.com/cnizzardini/13d0a072adb35a0d5817/raw/f315c97c7677845668a9c26e9093d0d550533b00/nfl_teams.csv',
                       index_col=0)
# team stats and rankings from each game per season

team19 = pd.read_csv("C:/Users/steph/OneDrive/Documents/Data Projects/2019_NFL_COMBINE.csv", index_col = 0)
team20 = pd.read_csv("C:/Users/steph/OneDrive/Documents/Data Projects/2020_NFL_COMBINE.csv", index_col = 0)
team21 = pd.read_csv("C:/Users/steph/OneDrive/Documents/Data Projects/2021_NFL_COMBINE.csv", index_col = 0)
team22 = pd.read_csv("C:/Users/steph/OneDrive/Documents/Data Projects/Fantasy/teamstats22.csv", index_col = 0)

# Incoming rookie data

rosters19 = nfl.import_rosters([2019], ['team', 'player_name','position', 'player_id', 
                                        'height', 'weight', 'years_exp'])

rosters20 = nfl.import_rosters([2020], ['team', 'player_name', 'position', 'player_id', 
                                        'height', 'weight', 'years_exp'])

rosters21 = nfl.import_rosters([2021], ['team', 'player_name', 'position', 'player_id', 'height', 
                                        'weight', 'years_exp'])

rosters22 = nfl.import_rosters([2022], ['team', 'player_name', 'position', 'player_id', 'height', 
                                        'weight', 'years_exp'])
rosters23 = nfl.import_rosters([2023], ['team', 'player_name', 'position', 'player_id', 'height', 
                                        'weight', 'years_exp'])

## Depth chart data

depth_chart = nfl.import_depth_charts([2019, 2020, 2021, 2022])

# seasonal data

season19 = nfl.import_seasonal_data([2019])
season20 = nfl.import_seasonal_data([2020])
season21 = nfl.import_seasonal_data([2021])
season22 = nfl.import_seasonal_data([2022])

# Injury Data?
injuries19 = nfl.import_injuries([2019])
injuries20 = nfl.import_injuries([2020])
injuries21 = nfl.import_injuries([2021])
injuries22 = nfl.import_injuries([2022])

# QBR data
# defaults are NFL and seasonal

qbr19 = nfl.import_qbr([2019])
qbr20 = nfl.import_qbr([2020])
qbr21 = nfl.import_qbr([2021])
qbr22 = nfl.import_qbr([2022])

# Next gen stat data

ngs_rushing = nfl.import_ngs_data('rushing', [2019, 2020, 2021, 2022])

ngs_receiving = nfl.import_ngs_data('receiving', [2019, 2020, 2021, 2022])

ngs_passing = nfl.import_ngs_data('passing', [2019, 2020, 2021, 2022])

# team wins

# this is actually just projected win total for gambling

team_wins = nfl.import_win_totals([2019, 2020, 2021, 2022])

# other data ideas:

# PFF Data (OL rankings, etc)
# contract data?


# clean data of team name, player name, and other descriptive data
# clean = nfl.clean_nfl_data(insert_Df)


C:\Users\steph\miniconda3\lib\site-packages\nfl_data_py\__init__.py:290: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  season_stats = all_stats.drop(['recent_team', 'week'], axis=1).groupby(
C:\Users\steph\miniconda3\lib\site-packages\nfl_data_py\__init__.py:309: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  szn = data.groupby(['player_id', 'season', 'season_type']).sum().reset_index().merge(
C:\Users\steph\miniconda3\lib\site-packages\nfl_data_py\__init__.py:290: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Ei

In [3]:
team22.head()

,Tot Yds & TO.1,Tot Yds & TO.2,Unnamed: 3,Unnamed: 4,Passing,Passing.1,Passing.2,Passing.3,Passing.4,Passing.5,...,Penalties.1,Penalties.2,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
Tot Yds & TO,,,,,,,,,,,,,,,,,,,,,
Rk,Tm,G,PF,Yds,Passing Ply,Y/P,TO,FL,1stD,Cmp,...,Yds,TD,Y/A,1stD,Pen,Yds,1stPy,Sc%,TO%,EXP
1,Kansas City Chiefs,17,496,7032,1094,6.4,23,11,408,435,...,1970,18,4.7,105,87,838,31,46.4,10.1,281
2,Philadelphia Eagles,17,477,6614,1124,5.9,19,10,385,350,...,2509,32,4.6,161,90,682,29,42.5,10.2,226.96
3,Dallas Cowboys,17,467,6034,1114,5.4,23,5,347,355,...,2298,24,4.3,129,104,842,24,41.5,10.8,112.34
4,Buffalo Bills,16,455,6361,1037,6.1,27,13,367,361,...,2232,15,5.2,128,92,709,27,45,15.2,157.46


The team data had to be from a different source due to the original only having the first 7 weeks of the season. Since we end up utilizing aggregated team data, rather than week to week, we will utilize this to join later on with the rest of the team data.

# Exploring Data Frames

## Depth Chart Data

One area that is interesting, and I'd hypothesize would have a large affect on fantasy scoring purposes, is where they are on the depth chart. 

In [4]:
rel_pos = ['QB', 'RB', 'WR', 'TE']

depth_chart = depth_chart.loc[depth_chart['position'].isin(rel_pos)]



In [5]:
# getting depth chart for week 1, the start of the season

depth_chart_start = depth_chart.loc[depth_chart['week'] == 1]

In [6]:
depth_chart_start['depth_team'] = depth_chart_start['depth_team'].astype(int)

C:\Users\steph\AppData\Local\Temp/ipykernel_16852/2753731436.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  depth_chart_start['depth_team'] = depth_chart_start['depth_team'].astype(int)


In [7]:
# function for mapping depth as a new column

def map_to_team(depth):
    if depth == 1.0:
        return 'starter'
    elif depth == 2.0:
        return '2nd team'
    elif depth == 3.0:
        return '3rd team'
    else:
        return '4th team or worse'

depth_chart_start['depth_lvl'] = depth_chart_start['depth_team'].apply(map_to_team)

C:\Users\steph\AppData\Local\Temp/ipykernel_16852/913454830.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  depth_chart_start['depth_lvl'] = depth_chart_start['depth_team'].apply(map_to_team)


In [8]:
depth_chart_start.head()

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,depth_lvl
19,2019.0,ATL,1.0,REG,2,Schaub,Matt,Matt,Offense,00-0022787,8,QB,SCH085186,QB,Matt Schaub,2nd team
33,2019.0,ATL,1.0,REG,1,Ryan,Matthew,Matt,Offense,00-0026143,2,QB,RYA238179,QB,Matt Ryan,starter
67,2019.0,ATL,1.0,REG,1,Jones,Quintorris,Julio,Offense,00-0027944,85,WR,JON640315,WR,Julio Jones,starter
134,2019.0,ATL,1.0,REG,2,Stocker,Lucas,Luke,Offense,00-0028042,35,TE,STO088140,TE,Luke Stocker,2nd team
157,2019.0,ATL,1.0,REG,1,Sanu,Mohamed,Mohamed,Offense,00-0029632,16,WR,SAN808447,WR,Mohamed Sanu,starter


In [9]:
## Let's keep only the most relevant columns

depth_chart_start = depth_chart_start[[ 'full_name','season', 'club_code', 'depth_team',
                                       'depth_lvl', 'gsis_id', 'depth_position']]

## Injury Data

In [10]:
# let's check out injury data

year = [19, 20, 21, 22]
injuries_filtered = []

for i in range(len(year)):
    season = year[i]
    injuries = f'injuries{season}'
    inj = globals()[injuries]
    fantasy_pos = {'QB', 'RB', 'WR', 'TE'}
    
    inj_filtered = inj[inj['position'].isin(fantasy_pos)]
    injuries_filtered.append(inj_filtered)

injuries19 = injuries_filtered[0]
injuries20 = injuries_filtered[1]
injuries21 = injuries_filtered[2]
injuries22 = injuries_filtered[3]

injuries = pd.concat([injuries19, injuries20, injuries21, injuries22])

In [11]:
injuries.loc[injuries['first_name'] == 'Tyreek']

,season,game_type,team,week,gsis_id,position,full_name,first_name,last_name,report_primary_injury,report_secondary_injury,report_status,practice_primary_injury,practice_secondary_injury,practice_status,date_modified
349,2019.0,REG,KC,2.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,Shoulder,None,Out,Shoulder,None,Did Not Participate In Practice,2019-09-13 12:48:28+00:00
651,2019.0,REG,KC,3.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,Shoulder,None,Out,Shoulder,None,Did Not Participate In Practice,2019-09-20 12:03:15+00:00
1002,2019.0,REG,KC,4.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,Shoulder,None,Out,Shoulder,None,Did Not Participate In Practice,2019-09-27 11:58:29+00:00
1310,2019.0,REG,KC,5.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,Shoulder,None,Out,Shoulder,None,Limited Participation in Practice,2019-10-04 12:38:41+00:00
1590,2019.0,REG,KC,6.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,Shoulder,None,Questionable,Shoulder,None,Limited Participation in Practice,2019-10-11 12:41:27+00:00
1889,2019.0,REG,KC,7.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,None,None,None,Shoulder,None,Full Participation in Practice,2019-10-16 10:41:11+00:00
3699,2019.0,REG,KC,13.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,None,None,None,Hamstring,None,Full Participation in Practice,2019-11-29 12:27:34+00:00
4040,2019.0,REG,KC,14.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,None,None,None,Hamstring,None,Full Participation in Practice,2019-12-04 14:04:32+00:00
4709,2019.0,REG,KC,16.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,None,None,None,Shoulder,None,Full Participation in Practice,2019-12-20 12:20:44+00:00
945,2020.0,REG,KC,4.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,None,None,None,Heel,None,Full Participation in Practice,2020-10-02 12:15:01+00:00


It seems these are the once a week injury reports. One way to aggregate this, perhaps, is to count the amount of 'Out's,'Questionable's, and both, grouped by season and player.

In [12]:
injuries['out_occured'] = injuries['report_status'].apply(lambda x: 1 if x == 'Out' else 0)
injuries['questionable_occured'] = injuries['report_status'].apply(lambda x: 1 if x == 'Questionable' else 0)

injuries['games_out'] = injuries.groupby(['gsis_id', 'season'])['out_occured'].transform('sum')
injuries['games_questionable'] = injuries.groupby(['gsis_id', 'season'])['questionable_occured'].transform('sum')
injuries['games_out_or_questionable'] = injuries['games_out'] + injuries['games_questionable']


injuries_by_season = injuries.groupby(['gsis_id', 'season'])\
                  [['full_name', 'games_out', 'games_questionable', 'games_out_or_questionable']]\
                 .agg({'full_name': 'first', 'games_out': 'mean',
                       'games_questionable': 'mean', 'games_out_or_questionable': 'mean'})\
                    .reset_index()

In [13]:
injuries_by_season

,gsis_id,season,full_name,games_out,games_questionable,games_out_or_questionable
0,00-0019596,2019.0,Tom Brady,0.0,1.0,1.0
1,00-0019596,2020.0,Tom Brady,0.0,0.0,0.0
2,00-0019596,2021.0,Tom Brady,0.0,0.0,0.0
3,00-0019596,2022.0,Tom Brady,0.0,0.0,0.0
4,00-0020531,2019.0,Drew Brees,5.0,1.0,6.0
...,...,...,...,...,...,...
1693,00-0038117,2022.0,Wan'Dale Robinson,4.0,2.0,6.0
1694,00-0038120,2022.0,Breece Hall,0.0,0.0,0.0
1695,00-0038124,2022.0,Christian Watson,2.0,4.0,6.0
1696,00-0038129,2022.0,Cade Otton,0.0,0.0,0.0


## Fantasy Data

Before diving into all the fantasy data, the data from 2022 was incomplete from the API utilized earlier. So, we will need to webscrape this data.

### Scraping FantasyData for 2022

In [14]:
### utilizing beautifulsoup to scrape ff22 data!

import requests
from bs4 import BeautifulSoup

# URL of the fantasy football page
url = 'https://www.pro-football-reference.com/years/2022/fantasy.htm'

# Send a GET request to the URL
response = requests.get(url)

# Create a BeautifulSoup object with the content of the response
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table containing the fantasy football data
table = soup.find('table', {'id': 'fantasy'})

# Extract the column names from the table headers
subheader_row = table.find('thead').find_all('tr')[1]
column_names = [subheader.text for subheader in subheader_row.find_all('th')][:-1]

# Find all rows in the table (excluding the header row)
rows = table.find_all('tr')[1:]

# Define a list to store the data
data = []

# Iterate over each row and extract the data
for row in rows:
    # Find all cells in the row
    cells = row.find_all('td')

    # Check if the row has the expected number of cells
    if len(cells) >= len(column_names):
        # Extract the data from the cells
        row_data = [cell.text for cell in cells[:len(column_names)]]

        # Append the data to the list
        data.append(row_data)

# Create a DataFrame from the data list
df = pd.DataFrame(data, columns = column_names)

# Save the DataFrame to a CSV file
df.to_csv('fantasy_data.csv', index=False)

In [15]:
# it seems the columns were off by 1
# shifting with code below, while dropping rank column

ff22 = df.shift(axis = 1).drop(columns = 'Rk')

#### Standardizing column names for all FF data

In [17]:
# looking at ff21 (similar columns as ff20 and ff19)
pd.set_option('display.max_columns', None)

ff21.head()

,Player,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost
0,Jonathan Taylor*+,IND,RB,22,17,17,51.0,40.0,0.0,0.0,0.0,1811.0,18.0,332.0,360.0,2.0,333.0,0.0,4.0,2.0
1,Cooper Kupp*+,LAR,WR,28,17,17,191.0,145.0,0.0,0.0,1.0,18.0,0.0,4.0,1947.0,16.0,295.0,0.0,0.0,0.0
2,Deebo Samuel*+,SFO,WR,25,16,15,121.0,77.0,24.0,1.0,2.0,365.0,8.0,59.0,1405.0,6.0,262.0,0.0,4.0,2.0
3,Josh Allen,BUF,QB,25,17,17,0.0,0.0,4407.0,36.0,646.0,763.0,6.0,122.0,0.0,0.0,403.0,15.0,8.0,3.0
4,Austin Ekeler,LAC,RB,26,16,16,94.0,70.0,0.0,0.0,0.0,911.0,12.0,206.0,647.0,8.0,274.0,0.0,4.0,3.0


In [18]:
# looking at ff22 -- these have different columns than ff19 - ff21

ff22.head()

,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,TD,Int,Att,Yds,Y/A,TD,Tgt,Rec,Yds,Y/R,TD,Fmb,FL,TD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank
0,Patrick Mahomes*+,KAN,QB,27,17,17,435,648,5250,41,12,61,358,5.87,4,1,1,6,6.00,0,5,0,4,1,2,416,417.4,435.4,428.9,136,1
1,Josh Jacobs*+,LVR,RB,24,17,17,0,0,0,0,0,340,1653,4.86,12,64,53,400,7.55,0,3,1,12,,,275,328.3,335.3,301.8,127,1
2,Christian McCaffrey*,2TM,RB,26,17,16,1,1,34,1,0,244,1139,4.67,8,108,85,741,8.72,5,1,0,13,,,271,356.4,362.4,313.9,123,2
3,Derrick Henry*,TEN,RB,28,16,16,2,2,4,1,0,349,1538,4.41,13,41,33,398,12.06,0,6,3,13,,,270,302.8,311.8,286.3,122,3
4,Justin Jefferson*+,MIN,WR,23,17,17,2,2,34,0,0,4,24,6.00,1,184,128,1809,14.13,8,0,0,9,1,,241,368.7,371.7,304.7,119,1


In [19]:
# due to main header not being included with, need to differentiate between identical column names like Yds or TD

# writing a function for duplicate names
def _maybe_dedup_names(self, names):
    if self.mangle_dupe_cols:
        names = list(names)  # to index
        counts = {}
        
        # getting index and col value
        for i, col in enumerate(names):
            cur_count = counts.get(col, 0)

            if cur_count > 0:
                names[i] = '%s.%d' % (col, cur_count)

            counts[col] = cur_count + 1

    return names

ff22.columns =pd.io.parsers.base_parser.ParserBase({'names':ff22.columns, 'usecols':None})._maybe_dedup_names(ff22.columns)

In [20]:
# since ff22 is from a dif source than ff19-ff21, we'll rename columns
# TD.3 is total touchdowns, incld special teams

ff22.rename(columns = {"FantPos":"Pos", "Att" : "PassingAtt", "Yds": "PassingYds",
            "TD": "PassingTD", "Att.1": "RushingAtt", "Yds.1": "RushingYds", "TD.1": "RushingTD", 
            "TD.1": "RushingTD", "Yds.2": "ReceivingYds", "TD.2": "ReceivingTD", 
                       "FantPt": "FantasyPoints", "TD.3": "TotalTds_22", "Fmb": "Fumbles",
                      "FL": "FumblesLost"}, inplace = True)

# ff22.drop('PTS/G', axis =1 ,inplace =True)

ff22.head()

,Player,Tm,Pos,Age,G,GS,Cmp,PassingAtt,PassingYds,PassingTD,Int,RushingAtt,RushingYds,Y/A,RushingTD,Tgt,Rec,ReceivingYds,Y/R,ReceivingTD,Fumbles,FumblesLost,TotalTds_22,2PM,2PP,FantasyPoints,PPR,DKPt,FDPt,VBD,PosRank
0,Patrick Mahomes*+,KAN,QB,27,17,17,435,648,5250,41,12,61,358,5.87,4,1,1,6,6.00,0,5,0,4,1,2,416,417.4,435.4,428.9,136,1
1,Josh Jacobs*+,LVR,RB,24,17,17,0,0,0,0,0,340,1653,4.86,12,64,53,400,7.55,0,3,1,12,,,275,328.3,335.3,301.8,127,1
2,Christian McCaffrey*,2TM,RB,26,17,16,1,1,34,1,0,244,1139,4.67,8,108,85,741,8.72,5,1,0,13,,,271,356.4,362.4,313.9,123,2
3,Derrick Henry*,TEN,RB,28,16,16,2,2,4,1,0,349,1538,4.41,13,41,33,398,12.06,0,6,3,13,,,270,302.8,311.8,286.3,122,3
4,Justin Jefferson*+,MIN,WR,23,17,17,2,2,34,0,0,4,24,6.00,1,184,128,1809,14.13,8,0,0,9,1,,241,368.7,371.7,304.7,119,1


## ADP 


### Scraping data on Fantasy Average Draft Positions (ADP)

In [21]:
# 2023 rankings (current year)

# URL of the website
rank_url = "https://fantasyfootballcalculator.com/adp/half-ppr/12-team/all"

# Send a GET request to the URL
response = requests.get(rank_url)

# Create a BeautifulSoup object with the content of the response
soup = BeautifulSoup(response.content, 'html.parser')

# Find the main table on the page
table = soup.find("table", class_="table adp")

# Find all rows in the table (excluding the header row)
rows = table.find_all("tr")[1:]

# Define lists to store the data
data = []
header = []

# Extract the column names from the header row
for th in table.find_all("th"):
    header.append(th.text.strip())

# Iterate over each row and extract the data
for row in rows:
    cells = row.find_all("td")
    row_data = [cell.text.strip() for cell in cells]
    data.append(row_data)

# Create a DataFrame from the data
ranks_23 = pd.DataFrame(data, columns=header)

# Print the DataFrame
print(ranks_23)

       #   Pick                 Name Pos Team Bye Overall Std.Dev   High  \
0      1   1.01     Justin Jefferson  WR  MIN  13     1.4     1.3   1.01   
1      2   1.02  Christian McCaffrey  RB   SF   9     2.4     1.8   1.01   
2      3   1.03        Austin Ekeler  RB  LAC   5     3.1     5.9   1.01   
3      4   1.05          Tyreek Hill  WR  MIA  10     4.6     3.1   1.02   
4      5   1.05         Travis Kelce  TE   KC  10     5.4     5.1   1.01   
..   ...    ...                  ...  ..  ...  ..     ...     ...    ...   
200  201  15.03        Greg Zuerlein  PK  NYJ   7   171.0    18.7   8.12   
201  202  15.04          Taysom Hill  TE   NO  11   171.9    22.4   8.07   
202  203  15.08            Zack Moss  RB  IND  11   176.2    16.5  12.07   
203  204  15.09          Jason Myers  PK  SEA   5   177.2    21.5   7.08   
204  205  16.02       Mecole Hardman  WR  NYJ   7   181.9    15.9  13.06   

       Low TimesDrafted Graph  
0     3.05          665        
1     4.08          768

In [22]:
# let's keep only relevant positions (QB, RB, WR, and TE)

ranks_23 = ranks_23.loc[ranks_23['Pos'].isin(['QB', 'RB', 'TE', 'WR'])]

# let's keep only relevant columns
ranks_23 = ranks_23[['#', 'Name', 'Pos', 'Team']]
ranks_23.rename(columns = {'#': 'adp'}, inplace = True)

#### Scraping all seasons

In [23]:
# let's do it for the rest of the seasons - 2019 thru 2022

# List of years
years = [2019, 2020, 2021, 2022]

# Define lists to store the data
all_data = []

# Iterate over each year
for year in years:
    # Construct the URL for the specific year
    url = f"https://fantasyfootballcalculator.com/adp/half-ppr/12-team/all/{year}"

    # Send a GET request to the URL
    response = requests.get(url)

    # Create a BeautifulSoup object with the content of the response
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the main table on the page
    table = soup.find("table", class_="table adp")

    # Find all rows in the table (excluding the header row)
    rows = table.find_all("tr")[1:]

    # Extract the column names from the header row
    header = [th.text.strip() for th in table.find_all("th")]

    # Extract the data from each row
    data = []
    for row in rows:
        cells = row.find_all("td")
        row_data = [cell.text.strip() for cell in cells]
        row_data.append(year) # add year as a column
        data.append(row_data)
        

    # Append the data to the overall list
    all_data.extend(data)

# adding an extra column
header.append("season")

# Create a DataFrame from the collected data
ranks_19_thru_22 = pd.DataFrame(all_data, columns= header)

# Print the DataFrame
print(ranks_19_thru_22)

       #   Pick                  Name  Pos Team Overall Std.Dev   High    Low  \
0      1   1.01        Saquon Barkley   RB  NYG     1.3     0.6   1.01   1.05   
1      2   1.03          Alvin Kamara   RB   NO     2.7     0.8   1.01   1.06   
2      3   1.03   Christian McCaffrey   RB  CAR     3.1     1.1   1.01   1.07   
3      4   1.04       Ezekiel Elliott   RB  DAL     3.8     1.1   1.01   1.06   
4      5   1.06         David Johnson   RB  ARI     6.2     1.6   1.02   1.10   
..   ...    ...                   ...  ...  ...     ...     ...    ...    ...   
745  120  12.12   LA Chargers Defense  DEF  LAC   144.1    14.9  10.01  14.09   
746  121  13.03       Harrison Butker   PK   KC   146.9    16.7   9.12  16.07   
747  122  13.07  Indianapolis Defense  DEF  IND   150.7    18.8  10.01  17.01   
748  123  13.07        Raheem Mostert   RB  MIA   151.1    17.1  10.03  15.12   
749  124  14.02           Dawson Knox   TE  BUF   157.9    26.9   8.01  17.12   

    TimesDrafted     season

In [24]:
# let's keep only relevant positions (QB, RB, WR, and TE)

ranks_19_thru_22_rel = ranks_19_thru_22.loc[ranks_19_thru_22['Pos'].isin(['QB', 'RB', 'TE', 'WR'])]

# let's keep only relevant columns
ranks_19_thru_22_rel = ranks_19_thru_22_rel[['#', 'Name', 'Pos', 'Team', 'season']]
ranks_19_thru_22_rel.rename(columns = {'#': 'adp'}, inplace = True)

# let's split by season
ranks_19 = ranks_19_thru_22_rel.loc[ranks_19_thru_22_rel['season'] == 2019]
ranks_20 = ranks_19_thru_22_rel.loc[ranks_19_thru_22_rel['season'] == 2020]
ranks_21 = ranks_19_thru_22_rel.loc[ranks_19_thru_22_rel['season'] == 2021]
ranks_22 = ranks_19_thru_22_rel.loc[ranks_19_thru_22_rel['season'] == 2022]


#### Cleaning player names

In [25]:
# some players have '+' or '*' in their name in some dfs to indicate if they're all-pro or pro-bowlers
# others may have Jr.s or SR.s and that can change year to year too
# we want to get rid of this so all names are just their names

import re

ranks_dfs = [ranks_19, ranks_20, ranks_21, ranks_22]

text_to_remove = ['+', '*', ' Jr.',' III', ' II', ' Sr.']

# to ensure we escape special characters so there are no errors
text_to_remove_escaped = [re.escape(text) for text in text_to_remove]

for df in ranks_dfs:
    df['Name'] = df['Name'].str.replace('|'.join(text_to_remove_escaped), '',regex= True )
    
ff21.head()

C:\Users\steph\AppData\Local\Temp/ipykernel_16852/41190835.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Name'] = df['Name'].str.replace('|'.join(text_to_remove_escaped), '',regex= True )


,Player,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost
0,Jonathan Taylor*+,IND,RB,22,17,17,51.0,40.0,0.0,0.0,0.0,1811.0,18.0,332.0,360.0,2.0,333.0,0.0,4.0,2.0
1,Cooper Kupp*+,LAR,WR,28,17,17,191.0,145.0,0.0,0.0,1.0,18.0,0.0,4.0,1947.0,16.0,295.0,0.0,0.0,0.0
2,Deebo Samuel*+,SFO,WR,25,16,15,121.0,77.0,24.0,1.0,2.0,365.0,8.0,59.0,1405.0,6.0,262.0,0.0,4.0,2.0
3,Josh Allen,BUF,QB,25,17,17,0.0,0.0,4407.0,36.0,646.0,763.0,6.0,122.0,0.0,0.0,403.0,15.0,8.0,3.0
4,Austin Ekeler,LAC,RB,26,16,16,94.0,70.0,0.0,0.0,0.0,911.0,12.0,206.0,647.0,8.0,274.0,0.0,4.0,3.0


In [26]:
ranks_19_thru_22.tail()

,#,Pick,Name,Pos,Team,Overall,Std.Dev,High,Low,TimesDrafted,,season
745,120,12.12,LA Chargers Defense,DEF,LAC,144.1,14.9,10.01,14.09,53,,2022
746,121,13.03,Harrison Butker,PK,KC,146.9,16.7,9.12,16.07,55,,2022
747,122,13.07,Indianapolis Defense,DEF,IND,150.7,18.8,10.01,17.01,52,,2022
748,123,13.07,Raheem Mostert,RB,MIA,151.1,17.1,10.03,15.12,57,,2022
749,124,14.02,Dawson Knox,TE,BUF,157.9,26.9,8.01,17.12,61,,2022


2022 data is limited here as well, only the first 124 adp players are here.

# Data Cleaning and Merging

This section is dedicated to the most important data for this project -- fantasy scoring data, team data, and roster data for each season.


Thoughts/notes:

Need to clean player name from *, + and/or both. Some of this was done earlier in the ADP section.

Merge in QBR data to QB data; ngs_rushing to qb and rb

need a dataset of predictor variables for final -- previous year's players (2021) plus rookies (2022) draft.

Need to bring in a dataset of rookies 

Need a dataset of team ranks from previous year (2021)

Need to group by position, as points scored and value is dependent on position.

## Fantasy Points Datasets

In [27]:
ff20.head()

,Player,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost
0,Derrick Henry,TEN,RB,26,16,16,31,19,0,0,0,2027,17,378,114,0,314.0,0,3,2
1,Alvin Kamara,NOR,RB,25,15,10,107,83,0,0,0,932,16,187,756,5,295.0,0,1,0
2,Dalvin Cook,MIN,RB,25,14,14,54,44,0,0,0,1557,16,312,361,1,294.0,0,5,3
3,Travis Kelce,KAN,TE,31,15,15,145,105,4,0,2,0,0,0,1416,11,208.0,0,1,1
4,Davante Adams,GNB,WR,28,14,14,149,115,0,0,0,0,0,0,1374,18,243.0,0,1,1


In [28]:
ff21.head()

,Player,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost
0,Jonathan Taylor*+,IND,RB,22,17,17,51.0,40.0,0.0,0.0,0.0,1811.0,18.0,332.0,360.0,2.0,333.0,0.0,4.0,2.0
1,Cooper Kupp*+,LAR,WR,28,17,17,191.0,145.0,0.0,0.0,1.0,18.0,0.0,4.0,1947.0,16.0,295.0,0.0,0.0,0.0
2,Deebo Samuel*+,SFO,WR,25,16,15,121.0,77.0,24.0,1.0,2.0,365.0,8.0,59.0,1405.0,6.0,262.0,0.0,4.0,2.0
3,Josh Allen,BUF,QB,25,17,17,0.0,0.0,4407.0,36.0,646.0,763.0,6.0,122.0,0.0,0.0,403.0,15.0,8.0,3.0
4,Austin Ekeler,LAC,RB,26,16,16,94.0,70.0,0.0,0.0,0.0,911.0,12.0,206.0,647.0,8.0,274.0,0.0,4.0,3.0


In [29]:
ff19.head()

,Player,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost
Unnamed: 0,,,,,,,,,,,,,,,,,,,,
0,Christian McCaffrey,CAR,RB,23.0,16.0,16.0,142.0,116.0,0.0,0.0,2.0,1387.0,15.0,287.0,1005.0,4.0,469.20,0.0,1.0,0.0
1,Lamar Jackson,BAL,QB,22.0,15.0,15.0,0.0,0.0,3127.0,36.0,401.0,1206.0,7.0,176.0,0.0,0.0,415.68,6.0,9.0,2.0
2,Derrick Henry,TEN,RB,25.0,15.0,15.0,24.0,18.0,0.0,0.0,0.0,1540.0,16.0,303.0,206.0,2.0,294.60,0.0,5.0,3.0
3,Aaron Jones,GNB,RB,25.0,16.0,16.0,68.0,49.0,0.0,0.0,0.0,1084.0,16.0,236.0,474.0,3.0,314.80,0.0,3.0,2.0
4,Ezekiel Elliott,DAL,RB,24.0,16.0,16.0,71.0,54.0,0.0,0.0,0.0,1357.0,12.0,301.0,420.0,2.0,311.70,0.0,3.0,2.0


### Cleaning Players Names

In [30]:
# some players have '+' or '*' in their name in some dfs to indicate if they're all-pro or pro-bowlers
# others may have Jr.s or SR.s and that can change year to year too
# we want to get rid of this so all names are just their names

ff_dfs = [ff22, ff21, ff20, ff19]

text_to_remove = ['+', '*', ' Jr.',' III', ' II', ' Sr.']

# to ensure we escape special characters so there are no errors
text_to_remove_escaped = [re.escape(text) for text in text_to_remove]

for df in ff_dfs:
    df['Player'] = df['Player'].str.replace('|'.join(text_to_remove_escaped), '',regex= True )
    
ff21.head()

,Player,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost
0,Jonathan Taylor,IND,RB,22,17,17,51.0,40.0,0.0,0.0,0.0,1811.0,18.0,332.0,360.0,2.0,333.0,0.0,4.0,2.0
1,Cooper Kupp,LAR,WR,28,17,17,191.0,145.0,0.0,0.0,1.0,18.0,0.0,4.0,1947.0,16.0,295.0,0.0,0.0,0.0
2,Deebo Samuel,SFO,WR,25,16,15,121.0,77.0,24.0,1.0,2.0,365.0,8.0,59.0,1405.0,6.0,262.0,0.0,4.0,2.0
3,Josh Allen,BUF,QB,25,17,17,0.0,0.0,4407.0,36.0,646.0,763.0,6.0,122.0,0.0,0.0,403.0,15.0,8.0,3.0
4,Austin Ekeler,LAC,RB,26,16,16,94.0,70.0,0.0,0.0,0.0,911.0,12.0,206.0,647.0,8.0,274.0,0.0,4.0,3.0


## Team data

Create Team Stats DFs for each season:

Due to limited 2022 data, need to supplement with a different data source. '19-'21 are via the API.

In [31]:
# ensure all teams are represented


team21['Home_team'].value_counts()

Los Angeles Rams        23
Cincinnati Bengals      23
San Francisco 49ers     22
Kansas City Chiefs      22
Tampa Bay Buccaneers    21
Buffalo Bills           21
Arizona Cardinals       20
Pittsburgh Steelers     20
Green Bay Packers       20
Dallas Cowboys          20
Philadelphia Eagles     20
Tennessee Titans        20
Oakland Raiders         20
New England Patriots    20
Houston Texans          18
Seattle Seahawks        18
New Orleans Saints      18
Baltimore Ravens        18
Carolina Panthers       18
New York Giants         18
Detroit Lions           18
New York Jets           18
Jacksonville Jaguars    18
Chicago Bears           18
Atlanta Falcons         18
Miami Dolphins          18
Indianapolis Colts      18
Los Angeles Chargers    18
Cleveland Browns        18
Denver Broncos          18
Minnesota Vikings       18
Name: Home_team, dtype: int64

The Washington Football Team (Commanders) are not represented in any of these team datasets.

In [32]:
# getting rid of bye week rows

teams = [team19, team20, team21]

for i in range(len(teams)):
    teams[i] = teams[i][teams[i].Opp != 'Bye Week']

In [33]:
team19 = teams[0]
team20 = teams[1]
team21 = teams[2]

Important to note, 2021 is first year with 18 weeks, rather than 17.

In [34]:
# must get rid of division, wild card, conf champ, and superbowl

reg1920 = np.arange(1, 18).astype(str)
reg21 = np.arange(1,19).astype(str)

team19_reg = team19.loc[team19['Week'].isin(reg1920)]
team20_reg = team20.loc[team20['Week'].isin(reg1920)]
team21_reg = team21.loc[team21['Week'].isin(reg21)]

team19_reg['Week'].value_counts()

1     31
2     31
3     31
17    31
16    31
15    31
14    31
13    31
4     29
8     29
5     29
11    27
12    27
9     27
7     27
6     27
10    26
Name: Week, dtype: int64

In [35]:
# keeping only relevant columns

relevant_team = ['Home_team',  'Unnamed: 5', 'Tm', '1stD', 'TotYd', 'PassY', 'RushY', 'TO',
           'Offense', 'Defense', 'Opp.1', 'TotYd.1', 'PassY.1', 'RushY.1', 'TO.1', 'Sp. Tms']

team19_reg = team19_reg[relevant_team]
team20_reg = team20_reg[relevant_team]
team21_reg = team21_reg[relevant_team]

# adding in columns to show season
team19_reg['season'] = '2019'
team20_reg['season'] = '2020'
team21_reg['season'] = '2021'

### Merging all seasons

In [36]:
# concatenating all years together

team_all =pd.concat([team19_reg, team20_reg, team21_reg])

team_all.head(20)

,Home_team,Unnamed: 5,Tm,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Opp.1,TotYd.1,PassY.1,RushY.1,TO.1,Sp. Tms,season
0,San Francisco 49ers,W,31.0,17.0,256.0,158.0,98.0,2.0,-7.51,19.12,17.0,295.0,174.0,121.0,4.0,1.10,2019
1,San Francisco 49ers,W,41.0,27.0,571.0,312.0,259.0,1.0,27.34,1.14,17.0,316.0,291.0,25.0,1.0,-6.64,2019
2,San Francisco 49ers,W,24.0,26.0,436.0,268.0,168.0,5.0,-3.72,10.20,20.0,239.0,160.0,79.0,2.0,0.12,2019
4,San Francisco 49ers,W,31.0,22.0,446.0,171.0,275.0,NaN,12.38,22.56,3.0,180.0,78.0,102.0,4.0,-10.50,2019
5,San Francisco 49ers,W,20.0,22.0,331.0,232.0,99.0,2.0,-3.54,25.23,7.0,157.0,48.0,109.0,1.0,-5.01,2019
6,San Francisco 49ers,W,9.0,17.0,283.0,146.0,137.0,1.0,-1.99,14.12,0.0,154.0,50.0,104.0,1.0,-0.61,2019
7,San Francisco 49ers,W,51.0,24.0,388.0,156.0,232.0,1.0,15.84,30.05,13.0,230.0,100.0,130.0,3.0,-7.14,2019
8,San Francisco 49ers,W,28.0,21.0,411.0,310.0,101.0,NaN,18.45,-13.71,25.0,357.0,204.0,153.0,NaN,0.88,2019
9,San Francisco 49ers,L,24.0,21.0,302.0,215.0,87.0,3.0,-17.73,17.69,27.0,336.0,189.0,147.0,4.0,-4.23,2019
10,San Francisco 49ers,W,36.0,26.0,442.0,408.0,34.0,2.0,8.92,-0.09,26.0,266.0,131.0,135.0,2.0,1.17,2019


In [37]:
# getting 'dummies' for the amount of wins, ties and losses each team has per season

team_all = pd.get_dummies(team_all, columns = ['Unnamed: 5']).groupby(['Home_team', 'season'], as_index = False).sum()

# from here, creating a new column that sums up wins and ties

team_all['team_wins'] = team_all['Unnamed: 5_W'] + 0.5 * team_all['Unnamed: 5_T']
team_all.head()

,Home_team,season,Tm,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Opp.1,TotYd.1,PassY.1,RushY.1,TO.1,Sp. Tms,Unnamed: 5_L,Unnamed: 5_T,Unnamed: 5_W,team_wins
0,Arizona Cardinals,2019,361.0,314.0,5467.0,3477.0,1990.0,18.0,64.36,-173.55,442.0,6432.0,4510.0,1922.0,17.0,19.85,10,1,5,5.5
1,Arizona Cardinals,2020,410.0,381.0,6153.0,3916.0,2237.0,21.0,103.62,-82.04,367.0,5631.0,3623.0,2008.0,21.0,12.58,8,0,8,8.0
2,Arizona Cardinals,2021,449.0,367.0,6352.0,4276.0,2076.0,15.0,137.75,-8.05,366.0,5597.0,3645.0,1952.0,27.0,-42.63,6,0,11,11.0
3,Atlanta Falcons,2019,381.0,383.0,6075.0,4714.0,1361.0,25.0,95.51,-98.34,399.0,5693.0,3918.0,1775.0,20.0,-22.36,9,0,7,7.0
4,Atlanta Falcons,2020,396.0,366.0,5895.0,4363.0,1532.0,18.0,129.77,-148.58,414.0,6374.0,4697.0,1677.0,21.0,1.13,12,0,4,4.0


In [38]:
## Aggregating to get each team's 

team_all = team_all.groupby(['Home_team', 'season']).mean().reset_index()

team_all.head()

,Home_team,season,Tm,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Opp.1,TotYd.1,PassY.1,RushY.1,TO.1,Sp. Tms,Unnamed: 5_L,Unnamed: 5_T,Unnamed: 5_W,team_wins
0,Arizona Cardinals,2019,361.0,314.0,5467.0,3477.0,1990.0,18.0,64.36,-173.55,442.0,6432.0,4510.0,1922.0,17.0,19.85,10.0,1.0,5.0,5.5
1,Arizona Cardinals,2020,410.0,381.0,6153.0,3916.0,2237.0,21.0,103.62,-82.04,367.0,5631.0,3623.0,2008.0,21.0,12.58,8.0,0.0,8.0,8.0
2,Arizona Cardinals,2021,449.0,367.0,6352.0,4276.0,2076.0,15.0,137.75,-8.05,366.0,5597.0,3645.0,1952.0,27.0,-42.63,6.0,0.0,11.0,11.0
3,Atlanta Falcons,2019,381.0,383.0,6075.0,4714.0,1361.0,25.0,95.51,-98.34,399.0,5693.0,3918.0,1775.0,20.0,-22.36,9.0,0.0,7.0,7.0
4,Atlanta Falcons,2020,396.0,366.0,5895.0,4363.0,1532.0,18.0,129.77,-148.58,414.0,6374.0,4697.0,1677.0,21.0,1.13,12.0,0.0,4.0,4.0


In [39]:
# rename columns to ensure we're clear what are defensive and offensive stats

team_all.rename(columns = {'Home_team': 'Team', 'Unnamed: 5_L': 'losses',  'Unnamed: 5_T': 'ties',\
                           'Unnamed: 5_W': 'wins', 'Tm': 'Pts_scored',
                          'Opp.1':'Pts_allowed', 'TotYd.1': 'Yds_alowed',
                          'PassY.1': 'PassYds_allowed', 'RushY.1': 'Rushyds_allowed',
                          "TO.1": "Team_takeaways"}, inplace = True)

In [40]:
team_all.head()

,Team,season,Pts_scored,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Pts_allowed,Yds_alowed,PassYds_allowed,Rushyds_allowed,Team_takeaways,Sp. Tms,losses,ties,wins,team_wins
0,Arizona Cardinals,2019,361.0,314.0,5467.0,3477.0,1990.0,18.0,64.36,-173.55,442.0,6432.0,4510.0,1922.0,17.0,19.85,10.0,1.0,5.0,5.5
1,Arizona Cardinals,2020,410.0,381.0,6153.0,3916.0,2237.0,21.0,103.62,-82.04,367.0,5631.0,3623.0,2008.0,21.0,12.58,8.0,0.0,8.0,8.0
2,Arizona Cardinals,2021,449.0,367.0,6352.0,4276.0,2076.0,15.0,137.75,-8.05,366.0,5597.0,3645.0,1952.0,27.0,-42.63,6.0,0.0,11.0,11.0
3,Atlanta Falcons,2019,381.0,383.0,6075.0,4714.0,1361.0,25.0,95.51,-98.34,399.0,5693.0,3918.0,1775.0,20.0,-22.36,9.0,0.0,7.0,7.0
4,Atlanta Falcons,2020,396.0,366.0,5895.0,4363.0,1532.0,18.0,129.77,-148.58,414.0,6374.0,4697.0,1677.0,21.0,1.13,12.0,0.0,4.0,4.0


### 2022 Data

2022 data was incomplete from original datasource, so, we are utilizing a different source to supplement

In [41]:
team22 = pd.read_csv("C:/Users/steph/OneDrive/Documents/Data Projects/Fantasy/teamstats22.csv", index_col = 0)

In [42]:
## Now for team22 to be brought in

team22.rename(columns = (team22.columns + '_' + team22.iloc[0, :]), inplace = True)


In [43]:
pd.set_option('display.max_columns', None)

team22.head()

,Tot Yds & TO.1_Tm,Tot Yds & TO.2_G,Unnamed: 3_PF,Unnamed: 4_Yds,Passing_Passing Ply,Passing.1_Y/P,Passing.2_TO,Passing.3_FL,Passing.4_1stD,Passing.5_Cmp,Passing.6_Att,Rushing_Yds,Rushing.1_TD,Rushing.2_Int,Rushing.3_NY/A,Rushing.4_1stD,Penalties_Att,Penalties.1_Yds,Penalties.2_TD,Unnamed: 20_Y/A,Unnamed: 21_1stD,Unnamed: 22_Pen,Unnamed: 23_Yds,Unnamed: 24_1stPy,Unnamed: 25_Sc%,Unnamed: 26_TO%,Unnamed: 27_EXP
Tot Yds & TO,,,,,,,,,,,,,,,,,,,,,,,,,,,
Rk,Tm,G,PF,Yds,Passing Ply,Y/P,TO,FL,1stD,Cmp,Att,Yds,TD,Int,NY/A,1stD,Att,Yds,TD,Y/A,1stD,Pen,Yds,1stPy,Sc%,TO%,EXP
1,Kansas City Chiefs,17,496,7032,1094,6.4,23,11,408,435,651,5062,41,12,7.5,272,417,1970,18,4.7,105,87,838,31,46.4,10.1,281
2,Philadelphia Eagles,17,477,6614,1124,5.9,19,10,385,350,536,4105,25,9,7.1,195,544,2509,32,4.6,161,90,682,29,42.5,10.2,226.96
3,Dallas Cowboys,17,467,6034,1114,5.4,23,5,347,355,556,3736,28,18,6.4,194,531,2298,24,4.3,129,104,842,24,41.5,10.8,112.34
4,Buffalo Bills,16,455,6361,1037,6.1,27,13,367,361,574,4129,35,14,6.8,212,430,2232,15,5.2,128,92,709,27,45,15.2,157.46


Many of these are still labeled incorrectly. Will rename the few we will use to match up with the column we will concatenate this with. Also, need to get rid of first row.

In [44]:
# getting rid of that first row

team22.drop(team22.index[0], inplace = True)

In [45]:
team_all.columns

Index(['Team', 'season', 'Pts_scored', '1stD', 'TotYd', 'PassY', 'RushY', 'TO',
       'Offense', 'Defense', 'Pts_allowed', 'Yds_alowed', 'PassYds_allowed',
       'Rushyds_allowed', 'Team_takeaways', 'Sp. Tms', 'losses', 'ties',
       'wins', 'team_wins'],
      dtype='object')

In [46]:
# creating columns to match our larger table

team22['season'] = 2022

# creating a dictionary to easily rename columns

cols_rename = {'Tot Yds & TO.1_Tm': 'Team', 'Unnamed: 3_PF': 'Pts_scored', 'Unnamed: 4_Yds': 'TotYd',
              'Rushing_Yds': 'PassY', 'Passing.2_TO': 'TO', 'Penalties.1_Yds': 'RushY', 'Passing.4_1stD': '1stD',
              'Tot Yds & TO.2_G': 'Team_games'}

team22_clean = team22.rename(columns = cols_rename)

In [47]:
team22_clean.tail()

,Team,Team_games,Pts_scored,TotYd,Passing_Passing Ply,Passing.1_Y/P,TO,Passing.3_FL,1stD,Passing.5_Cmp,Passing.6_Att,PassY,Rushing.1_TD,Rushing.2_Int,Rushing.3_NY/A,Rushing.4_1stD,Penalties_Att,RushY,Penalties.2_TD,Unnamed: 20_Y/A,Unnamed: 21_1stD,Unnamed: 22_Pen,Unnamed: 23_Yds,Unnamed: 24_1stPy,Unnamed: 25_Sc%,Unnamed: 26_TO%,Unnamed: 27_EXP,season
Tot Yds & TO,,,,,,,,,,,,,,,,,,,,,,,,,,,,
31,Indianapolis Colts,17,289,5298,1103,4.8,34,14,310,398,604,3432,17,20,5.2,193,439,1866,8,4.3,91,89,794,26,29.2,16.9,-110.46,2022
32,Denver Broncos,17,287,5527,1078,5.1,24,9,300,345,571,3592,18,15,5.7,166,444,1935,11,4.4,105,113,970,29,28.6,10.6,-41.62,2022
NaN,Avg Team,NaN,370.6,5760.4,1066.8,5.4,22.2,9.1,333.3,362.7,564.7,3701.2,23.4,13.1,6.1,190.2,461.6,2059.2,15.2,4.5,115.2,94.8,780.5,27.9,36.2,11.3,52.1,2022
NaN,League Total,NaN,11860,184332,34136,5.4,710,292,10666,11605,18069,118438,750,418,6.1,6087,14770,65894,487,4.5,3686,3032,24976,893,36.2,11.3,NaN,2022
NaN,Avg Tm/G,NaN,21.9,340.1,63,5.4,1.3,0.5,19.7,21.4,33.3,218.5,1.4,0.8,6.1,11.2,27.3,121.6,0.9,4.5,6.8,5.6,46.1,1.6,36.2,11.3,NaN,2022


In [48]:
# getting rid of last 3 rows, which are aggregates of all teams

team22_clean = team22_clean.drop(team22_clean.index[-3:])

In [49]:
team22_clean = team22_clean[['Team', 'season', 'Team_games', 'Pts_scored', 'TotYd', 'TO', '1stD', 'PassY', 'RushY']]

In [50]:
# bringing in 22 data with the rest
team_all = pd.concat([team_all, team22_clean], axis = 0, ignore_index = True)

In [51]:
nflteams.head()

,Name,Abbreviation,Conference,Division
ID,,,,
1,Arizona Cardinals,ARI,NFC,West
2,Atlanta Falcons,ATL,NFC,South
3,Baltimore Ravens,BAL,AFC,North
4,Buffalo Bills,BUF,AFC,East
5,Carolina Panthers,CAR,NFC,South


In [52]:
# bringing in Abbreviations

team_all_collective = team_all.merge(nflteams[['Name','Abbreviation']], left_on = 'Team', right_on = 'Name').drop(columns = 'Name')

team_all_collective.head()

,Team,season,Pts_scored,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Pts_allowed,Yds_alowed,PassYds_allowed,Rushyds_allowed,Team_takeaways,Sp. Tms,losses,ties,wins,team_wins,Team_games,Abbreviation
0,Arizona Cardinals,2019,361.0,314.0,5467.0,3477.0,1990.0,18.0,64.36,-173.55,442.0,6432.0,4510.0,1922.0,17.0,19.85,10.0,1.0,5.0,5.5,NaN,ARI
1,Arizona Cardinals,2020,410.0,381.0,6153.0,3916.0,2237.0,21.0,103.62,-82.04,367.0,5631.0,3623.0,2008.0,21.0,12.58,8.0,0.0,8.0,8.0,NaN,ARI
2,Arizona Cardinals,2021,449.0,367.0,6352.0,4276.0,2076.0,15.0,137.75,-8.05,366.0,5597.0,3645.0,1952.0,27.0,-42.63,6.0,0.0,11.0,11.0,NaN,ARI
3,Arizona Cardinals,2022,340,326,5499,3626,1873,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,ARI
4,Atlanta Falcons,2019,381.0,383.0,6075.0,4714.0,1361.0,25.0,95.51,-98.34,399.0,5693.0,3918.0,1775.0,20.0,-22.36,9.0,0.0,7.0,7.0,NaN,ATL


## Roster data

Moving onto roster data for each team

In [53]:
rosters19.sample(n=8)

,team,player_name,position,player_id,height,weight,years_exp
552,CHI,Mitchell Trubisky,QB,00-0033869,74.0,215.0,2.0
233,BAL,Josh Bynes,LB,00-0028278,73.0,235.0,8.0
1527,KC,Deon Yelder,TE,00-0034319,76.0,255.0,1.0
712,CLE,Willie Harvey,LB,00-0035177,71.0,230.0,0.0
1269,IND,Gerri Green,DL,00-0034997,76.0,250.0,NaN
2058,NE,Scooby Wright,LB,00-0032979,72.0,240.0,NaN
1550,LA,Adonis Alexander,DB,00-0034840,75.0,205.0,NaN
3041,WAS,Treyvon Hester,DL,00-0033976,74.0,304.0,2.0


In [54]:
## look at 2023 data

rosters23.sample(n=8)

,team,player_name,position,player_id,height,weight,years_exp
1051,GB,Preston Smith,LB,00-0031935,77.0,265.0,8
510,CHI,Mykal Walker,LB,00-0036300,75.0,219.0,3
1783,MIA,Parry Nickerson,DB,00-0034274,70.0,180.0,5
2335,PHI,Darius Slay,DB,00-0030062,72.0,190.0,10
2816,TEN,T.K. McLendon,DL,00-0038850,76.0,270.0,0
666,CLE,Ronnie Hickman,DB,00-0038715,73.0,205.0,0
971,DET,Colby Sorsdal,OL,00-0038592,78.0,305.0,0
2110,NO,Ellis Merriweather,RB,00-0038965,74.0,225.0,0


In [55]:
# We must get rid of non-offensive, fantasy irrelevant positions

# for this analysis, only focused on qb, rb, wr, te

ff_positions = ['QB', 'RB', 'WR', 'TE']


rosters19_rel = rosters19.loc[rosters19['position'].isin(ff_positions)]
rosters20_rel = rosters20.loc[rosters20['position'].isin(ff_positions)]
rosters21_rel = rosters21.loc[rosters21['position'].isin(ff_positions)]
rosters22_rel = rosters22.loc[rosters22['position'].isin(ff_positions)]
rosters23_rel = rosters23.loc[rosters23['position'].isin(ff_positions)]

In [56]:
# since we may be joining roster data with data that doesn't have the player keys assigned
# will have to clean player_names
# may have Jr.s or SR.s and that can change year to year too
# we want to get rid of this so all names are just their names

roster_dfs = [rosters19_rel, rosters20_rel, rosters21_rel, rosters22_rel, rosters23_rel]

text_to_remove = ['+', '*', ' Jr.',' III', ' II', ' Sr.']

# to ensure we escape special characters so there are no errors
text_to_remove_escaped = [re.escape(text) for text in text_to_remove]

for df in roster_dfs:
    df['player_name'] = df['player_name'].str.replace('|'.join(text_to_remove_escaped), '',regex= True )
    
rosters21_rel.head()

C:\Users\steph\AppData\Local\Temp/ipykernel_16852/1784432442.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['player_name'] = df['player_name'].str.replace('|'.join(text_to_remove_escaped), '',regex= True )


,team,player_name,position,player_id,height,weight,years_exp
57,ARI,Colt McCoy,QB,00-0027688,73.0,212,11
58,ARI,Kyler Murray,QB,00-0035228,70.0,207,2
59,ARI,Trace McSorley,QB,00-0035146,72.0,202,2
60,ARI,James Conner,RB,00-0033553,73.0,233,4
61,ARI,Chase Edmonds,RB,00-0034681,69.0,210,3


## Merging Data Frames So Far

Much of initial analysis and project has been without using the season data, rather fantasy data and bringing in next gen stats, etc. However, the season_data is possibly a cleaner way of going about it. 

In [57]:
ff19_r = rosters19_rel.merge(ff19, how = 'left', left_on = 'player_name', \
                             right_on = 'Player').reset_index(drop = True). \
                            merge(ranks_19[['adp', 'Name']], how = 'left', 
                                  left_on = 'player_name',right_on = 'Name')
ff19_r.drop(columns = 'Player', inplace = True)


ff19_r.sample(5)

,team,player_name,position,player_id,height,weight,years_exp,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost,adp,Name
568,LV,Zay Jones,WR,00-0033891,74.0,200.0,2.0,2TM,WR,24.0,15.0,9.0,45.0,27.0,0.0,0.0,0.0,3.0,0.0,1.0,216.0,0.0,48.9,0.0,0.0,0.0,NaN,NaN
427,JAX,Jeremy McNichols,RB,00-0033955,69.0,205.0,2.0,JAX,0,24.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
987,WAS,Steven Sims,WR,00-0034928,70.0,176.0,0.0,WAS,WR,22.0,16.0,2.0,56.0,34.0,0.0,0.0,1.0,85.0,1.0,9.0,310.0,4.0,103.5,0.0,1.0,0.0,NaN,NaN
701,NYG,Jon Hilliman,RB,00-0035013,72.0,226.0,0.0,NYG,RB,24.0,3.0,1.0,4.0,3.0,0.0,0.0,0.0,91.0,0.0,30.0,1.0,0.0,8.2,0.0,2.0,2.0,NaN,NaN
397,IND,Jack Doyle,TE,00-0030181,78.0,262.0,6.0,IND,TE,29.0,16.0,16.0,72.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,448.0,4.0,111.8,0.0,1.0,0.0,NaN,NaN


In [58]:
ff20_r = rosters20_rel.merge(ff20, how = 'left', left_on = 'player_name', \
                             right_on = 'Player', suffixes = ('_20r', '_20')).\
                    reset_index(drop = True).merge(ranks_20[['adp', 'Name']], \
                                            how = 'left', left_on = 'player_name',right_on = 'Name')
ff20_r.drop(columns = 'Player', inplace = True)

ff20_r.sample(5)

,team,player_name,position,player_id,height,weight,years_exp,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost,adp,Name
433,LA,Trishton Jackson,WR,00-0035934,73.0,191,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
373,JAX,Tyler Davis,TE,00-0036250,76.0,250,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
462,LV,Isaiah Crowell,RB,00-0030656,NaN,225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
704,SEA,Travis Homer,RB,00-0035594,70.0,202,1.0,SEA,RB,22.0,9.0,0.0,10.0,9.0,0.0,0.0,0.0,88.0,0.0,25.0,90.0,1.0,24.0,0.0,0.0,0.0,NaN,NaN
685,PIT,Charles Jones,TE,00-0035553,76.0,255,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
ff20_r.shape

(840, 28)

In [60]:
# merging with rosters and adp ranks

ff21_r = rosters21_rel.merge(ff21, how = 'left', left_on = 'player_name', \
                             right_on = 'Player', suffixes = ('_21r', '_21')) \
                        .reset_index(drop = True).merge(ranks_21[['adp', 'Name']], \
                                            how = 'left', left_on = 'player_name',right_on = 'Name')
ff21_r.drop(columns = 'Player', inplace = True)

ff21_r.sample(5)

,team,player_name,position,player_id,height,weight,years_exp,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost,adp,Name
28,ATL,Cordarrelle Patterson,RB,00-0030578,74.0,238,8,ATL,RB,30.0,16.0,13.0,69.0,52.0,0.0,0.0,1.0,618.0,6.0,153.0,548.0,5.0,183.0,0.0,2.0,0.0,NaN,NaN
413,KC,Mathew Sexton,WR,00-0036471,NaN,175,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
546,MIN,Myron Mitchell,WR,00-0036815,73.0,180,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
434,LA,Cooper Kupp,WR,00-0033908,74.0,208,4,LAR,WR,28.0,17.0,17.0,191.0,145.0,0.0,0.0,1.0,18.0,0.0,4.0,1947.0,16.0,295.0,0.0,0.0,0.0,48,Cooper Kupp
488,LV,Dillon Stoner,WR,00-0036526,72.0,198,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
ff22_r = rosters22_rel.merge(ff22, how = 'left', left_on = 'player_name', 
                             right_on = 'Player', suffixes = ('_22r', '_22'))\
            .reset_index(drop = True).merge(ranks_22[['adp', 'Name']], \
                                            how = 'left', left_on = 'player_name',right_on = 'Name')
ff22_r.drop(columns = 'Player', inplace = True)

ff22_r.sample(5)

,team,player_name,position,player_id,height,weight,years_exp,Tm,Pos,Age,G,GS,Cmp,PassingAtt,PassingYds,PassingTD,Int,RushingAtt,RushingYds,Y/A,RushingTD,Tgt,Rec,ReceivingYds,Y/R,ReceivingTD,Fumbles,FumblesLost,TotalTds_22,2PM,2PP,FantasyPoints,PPR,DKPt,FDPt,VBD,PosRank,adp,Name
458,LV,Jarrett Stidham,QB,00-0035264,74.0,214.0,3,LVR,QB,26,5,2,53,83,656,4,3,14,84,6.00,0,0,0,0,,0,3,1,0,,,43,42.6,49.6,45.6,,43,NaN,NaN
695,SEA,Laquon Treadwell,WR,00-0032951,74.0,215.0,6,SEA,WR,27,6,0,0,0,0,0,0,0,0,,0,10,6,42,7.00,0,0,0,0,,,4,10.2,10.2,7.2,,176,NaN,NaN
276,DET,Corey Sutton,WR,00-0037444,NaN,195.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
757,TEN,Dontrell Hilliard,RB,00-0034253,71.0,202.0,4,TEN,RB,27,12,1,0,0,0,0,0,22,145,6.59,0,26,21,177,8.43,4,0,0,4,,,56,77.2,83.2,66.7,,59,NaN,NaN
323,HOU,Nico Collins,WR,00-0036554,76.0,222.0,1,HOU,WR,23,10,7,0,0,0,0,0,0,0,,0,66,37,481,13.00,2,0,0,2,,,60,97.1,100.1,78.6,,77,NaN,NaN


In [62]:
# making sure all columns are lower case

ff19_r.columns = map(str.lower, ff19_r.columns)
ff20_r.columns = map(str.lower, ff20_r.columns)
ff21_r.columns = map(str.lower, ff21_r.columns)
ff22_r.columns = map(str.lower, ff22_r.columns)


In [63]:
ff19_r['season'] = 2019
ff20_r['season'] = 2020
ff21_r['season'] = 2021
ff22_r['season'] = 2022

In [64]:
# drop duplicates from all

ff22_r = ff22_r.drop_duplicates(subset = 'player_id')
ff21_r = ff21_r.drop_duplicates(subset = 'player_id')
ff20_r = ff20_r.drop_duplicates(subset =  'player_id')
ff19_r = ff19_r.drop_duplicates(subset = 'player_id')


In [65]:
# ff19_r = ff19_r.merge(depth_19,how = 'left', left_on = 'player_id',right_on = 'gsis_id')

# ff20_r = ff20_r.merge(depth_20,how = 'left', left_on = 'player_id', right_on = 'gsis_id')
# ff21_r = ff21_r.merge(depth_21,how = 'left', left_on = 'player_id',right_on = 'gsis_id')

# ff22_r = ff22_r.merge(depth_22,how = 'left', left_on = 'player_id', right_on = 'gsis_id')

In [66]:
ff22_r.head()

,team,player_name,position,player_id,height,weight,years_exp,tm,pos,age,g,gs,cmp,passingatt,passingyds,passingtd,int,rushingatt,rushingyds,y/a,rushingtd,tgt,rec,receivingyds,y/r,receivingtd,fumbles,fumbleslost,totaltds_22,2pm,2pp,fantasypoints,ppr,dkpt,fdpt,vbd,posrank,adp,name,season
0,ARI,Colt McCoy,QB,00-0027688,73.0,212.0,12,ARI,QB,36,4,3,90,132,780,1,3,14,36,2.57,0,0,0,0,,0,2,0,0,,,33,32.8,38.8,35.8,,50,NaN,NaN,2022
1,ARI,Kyler Murray,QB,00-0035228,70.0,207.0,3,ARI,QB,25,11,11,259,390,2368,14,7,67,418,6.24,3,0,0,0,,0,8,2,3,1,3,201,200.5,215.5,207.5,,18,56,Kyler Murray,2022
2,ARI,Trace McSorley,QB,00-0035146,72.0,202.0,3,ARI,QB,27,6,1,45,83,412,0,5,15,61,4.07,0,0,0,0,,0,2,2,0,,,9,8.6,18.6,13.6,,68,NaN,NaN,2022
3,ARI,David Blough,QB,00-0035040,73.0,200.0,3,ARI,QB,27,2,2,38,58,402,2,2,4,5,1.25,0,0,0,0,,0,1,1,0,,,19,18.6,24.6,20.6,,54,NaN,NaN,2022
4,ARI,James Morgan,QB,00-0036301,76.0,213.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022


### merging all 4 years together

Creating a 'tall' dataframe, and from there, we will pivot to make a wide one for ML purposes

In [67]:
# creating a collective DF as well

ff_all_collective = pd.concat([ff19_r, ff20_r, ff21_r, ff22_r])

ff_all_collective.columns


Index(['team', 'player_name', 'position', 'player_id', 'height', 'weight',
       'years_exp', 'tm', 'pos', 'age', 'g', 'gs', 'tgt', 'rec', 'passingyds',
       'passingtd', 'passingatt', 'rushingyds', 'rushingtd', 'rushingatt',
       'receivingyds', 'receivingtd', 'fantasypoints', 'int', 'fumbles',
       'fumbleslost', 'adp', 'name', 'season', 'cmp', 'y/a', 'y/r',
       'totaltds_22', '2pm', '2pp', 'ppr', 'dkpt', 'fdpt', 'vbd', 'posrank'],
      dtype='object')

After the merges, let's ensure the columns are all relevant and no duplicate information

In [68]:
ff_all_collective.shape

(3467, 40)

There are still plenty of columns, and plenty to possibly get rid of and clean well before modeling. Before that, we will continue bringing data in

# Data Cleaning and Merging Part 2

There is more data to clean and merge. The below will be on newer resources available as well as some creation of features where possible. Below are the areas and the questions I have for each.

QBR Dataset
       - not only for the QB themselves, but for the QB of the team the player will be on in current season
Team performance
     - how did the team perform that the player is currently on last year?
Next gen stats
    - how did the player perform by next gen stats?


Notes:
Other areas to consider/add features from :
    - QBR data set (how did current qb ('21) do in 20 and 19?)
    - Any way to measure OL play? Maybe from RB Next Gen stats, or from next gen stats as a whole prior to filtering positions?

## QBR Data

### Initial exploration + Cleaning
Getting this ready to merge with the rest of the data

In [69]:
qbr19 = nfl.import_qbr([2019])
qbr20 = nfl.import_qbr([2020])
qbr21 = nfl.import_qbr([2021])
qbr22 = nfl.import_qbr([2022])

In [70]:
qbr19.head()

,season,season_type,game_week,team_abb,player_id,name_short,rank,qbr_total,pts_added,qb_plays,epa_total,pass,run,exp_sack,penalty,qbr_raw,sack,name_first,name_last,name_display,headshot_href,team,qualified
937,2019,Regular,Season Total,BAL,3916387,L. Jackson,1.0,83.0,66.7,613,103.7,55.0,39.1,0,2.2,82.3,-7.4,Lamar,Jackson,Lamar Jackson,https://a.espncdn.com/i/headshots/nfl/players/...,Ravens,True
938,2019,Regular,Season Total,KC,3139477,P. Mahomes,2.0,77.7,55.8,585,97.3,71.6,14.3,0,5.0,79.4,-6.5,Patrick,Mahomes,Patrick Mahomes,https://a.espncdn.com/i/headshots/nfl/players/...,Chiefs,True
939,2019,Regular,Season Total,NO,2580,D. Brees,3.0,73.3,33.7,419,62.6,53.1,1.6,0,1.9,75.7,-6.0,Drew,Brees,Drew Brees,https://a.espncdn.com/i/headshots/nfl/players/...,Saints,True
940,2019,Regular,Season Total,DAL,2577417,D. Prescott,4.0,71.9,48.1,690,93.1,70.7,10.0,0,2.6,72.8,-9.7,Dak,Prescott,Dak Prescott,https://a.espncdn.com/i/headshots/nfl/players/...,Cowboys,True
941,2019,Regular,Season Total,SEA,14881,R. Wilson,5.0,71.5,43.0,674,90.9,58.3,10.6,0,1.5,71.2,-20.6,Russell,Wilson,Russell Wilson,https://a.espncdn.com/i/headshots/nfl/players/...,Seahawks,True


player_id isn't the same between QBR dataset and the rest of our data. Need to join on different keys.

Also, let's see what qbs are "qualified" and see what other columns we can get rid of in 2019 data

In [71]:
qbr19['qualified'].value_counts()

True     42
False    39
Name: qualified, dtype: int64

In [72]:
qbr19['qb_plays'].describe()

count     81.000000
mean     249.000000
std      290.395592
min        1.000000
25%        1.000000
50%       76.000000
75%      588.000000
max      772.000000
Name: qb_plays, dtype: float64

In [73]:
qbr20['qb_plays'].describe()

count     84.000000
mean     246.595238
std      275.832021
min        1.000000
25%        3.000000
50%       81.500000
75%      548.750000
max      763.000000
Name: qb_plays, dtype: float64

In [74]:
# let's get rid of playoffs

qbr19 = qbr19.loc[qbr19['season_type'] == 'Regular']
qbr20 = qbr20.loc[qbr20['season_type'] == 'Regular']
qbr21 = qbr21.loc[qbr21['season_type'] == 'Regular']
qbr22 = qbr22.loc[qbr22['season_type'] == 'Regular']

In [75]:
# let's lessen the columns

qbr19.columns

Index(['season', 'season_type', 'game_week', 'team_abb', 'player_id',
       'name_short', 'rank', 'qbr_total', 'pts_added', 'qb_plays', 'epa_total',
       'pass', 'run', 'exp_sack', 'penalty', 'qbr_raw', 'sack', 'name_first',
       'name_last', 'name_display', 'headshot_href', 'team', 'qualified'],
      dtype='object')

In [76]:
qbr19.describe()

,season,player_id,rank,qbr_total,pts_added,qb_plays,epa_total,pass,run,exp_sack,penalty,qbr_raw,sack
count,59.0,5.900000e+01,30.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.0,59.000000,59.000000,59.000000
mean,2019.0,1.636528e+06,15.500000,70.572881,8.667797,325.864407,33.310169,20.596610,3.528814,0.0,1.740678,70.727119,-7.452542
std,0.0,1.650638e+06,8.799491,20.722109,17.422908,305.732950,33.787866,22.819652,6.920735,0.0,1.957049,20.794129,7.358302
min,2019.0,2.330000e+03,1.000000,36.200000,-20.100000,1.000000,0.100000,-0.800000,-7.300000,0.0,-0.600000,36.800000,-20.600000
25%,2019.0,1.297200e+04,8.250000,54.650000,0.350000,1.500000,0.800000,0.600000,0.000000,0.0,0.000000,53.950000,-13.700000
50%,2019.0,2.468609e+06,16.000000,64.100000,1.100000,336.000000,25.500000,9.400000,0.700000,0.0,1.100000,65.600000,-6.500000
75%,2019.0,3.115732e+06,22.750000,97.350000,17.150000,626.000000,60.000000,42.250000,5.850000,0.0,3.000000,96.500000,0.000000
max,2019.0,4.038524e+06,30.000000,100.000000,66.700000,772.000000,103.700000,71.600000,39.100000,0.0,6.400000,100.000000,0.000000


### Merging all years together

In [77]:
# dropping columns from DFs

qbr_irrel = ['season_type', 'game_week', 'name_short', 'rank', 'penalty', 'name_first', 'name_last',
                     'headshot_href', 'team', 'qualified']

qbr19.drop(columns = qbr_irrel, inplace = True)
qbr20.drop(columns = qbr_irrel, inplace = True)
qbr21.drop(columns = qbr_irrel, inplace = True)
qbr22.drop(columns = qbr_irrel, inplace = True)

# creating collective of all qbr data
qbr_collective = pd.concat([qbr19, qbr20, qbr21, qbr22])


## Next Gen Data

With NextGen data, we want to have table for each position with all RBs, all QBs, all WRs and all TEs

### Running Backs

In [78]:
ngs_rushing.sample(5)

,season,season_type,week,player_display_name,player_position,team_abbr,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,expected_rush_yards,rush_yards_over_expected,avg_rush_yards,rush_yards_over_expected_per_att,rush_pct_over_expected,rush_touchdowns,player_gsis_id,player_first_name,player_last_name,player_jersey_number,player_short_name
3064,2021,REG,2,Ty Johnson,RB,NYJ,3.937200,41.666667,2.496273,12,50,42.355321,5.644679,4.166667,0.513153,0.363636,0,00-0035537,Ty,Johnson,25,T.Johnson
3113,2021,REG,4,Tony Pollard,RB,DAL,2.659104,40.000000,2.882000,10,67,36.191084,30.808916,6.700000,3.080892,0.600000,0,00-0035261,Tony,Pollard,20,T.Pollard
3604,2022,REG,0,David Montgomery,RB,CHI,4.102047,23.880597,2.781601,201,801,846.546044,-52.546044,3.985075,-0.264050,0.376884,5,00-0035685,David,Montgomery,32,D.Montgomery
3019,2021,REG,1,Darrell Henderson,RB,LAR,3.601286,25.000000,2.713333,16,70,64.368123,-3.368123,4.375000,-0.224542,0.400000,1,00-0035664,Darrell,Henderson,27,D.Henderson
3965,2022,REG,12,Saquon Barkley,RB,NYG,4.276923,36.363636,2.425800,11,39,43.689684,-4.689684,3.545455,-0.426335,0.272727,1,00-0034844,Saquon,Barkley,26,S.Barkley


In [79]:
ngs_rushing.columns

Index(['season', 'season_type', 'week', 'player_display_name',
       'player_position', 'team_abbr', 'efficiency',
       'percent_attempts_gte_eight_defenders', 'avg_time_to_los',
       'rush_attempts', 'rush_yards', 'expected_rush_yards',
       'rush_yards_over_expected', 'avg_rush_yards',
       'rush_yards_over_expected_per_att', 'rush_pct_over_expected',
       'rush_touchdowns', 'player_gsis_id', 'player_first_name',
       'player_last_name', 'player_jersey_number', 'player_short_name'],
      dtype='object')

In [80]:
# we only want regular season

ngs_rushing = ngs_rushing.loc[ngs_rushing['season_type'] == 'REG']

# jersey number, first last name, and short name don't matter
to_drop = ['week', 'season_type', 'player_first_name', 'player_last_name', 'player_jersey_number', 'player_short_name']

ngs_rushing.drop(columns = to_drop, inplace = True)

In [81]:
ngs_rushing['player_position'].unique()

array(['RB', 'HB'], dtype=object)

In [82]:
ngs_rushing['player_position'] = 'RB'

In [83]:
ngs_rb_collective = ngs_rushing.groupby(['season','player_display_name'], as_index = False).mean(numeric_only = True)


In [84]:
ngs_avg_stats_19 = ngs_rb_collective[ngs_rb_collective.season == 2019]
ngs_avg_stats_20 = ngs_rb_collective[ngs_rb_collective.season == 2020]
ngs_avg_stats_21 = ngs_rb_collective[ngs_rb_collective.season == 2021]
ngs_avg_stats_22 = ngs_rb_collective[ngs_rb_collective.season == 2022]

In [85]:
# merging all together to save into one larger DF with columns for each seasons stats

ngs_avg_stats_rb = ngs_avg_stats_19.merge(ngs_avg_stats_20, on = 'player_display_name', how = 'outer', suffixes = ('_19', '_20'))
ngs_avg_stats_rb = ngs_avg_stats_rb.merge(ngs_avg_stats_21, on = 'player_display_name', how = 'outer', suffixes = ('', '_21')) \
                    .merge(ngs_avg_stats_22, on = 'player_display_name', how = 'outer', suffixes = ('', '_22'))

In [86]:
# bringing player id back in

ngs_avg_collective_rb = ngs_rb_collective.merge(ngs_rushing[['player_display_name', 'player_gsis_id']], 
                                                        on = 'player_display_name', how = 'inner').drop_duplicates()

ngs_avg_stats_rb = ngs_avg_stats_rb.merge(ngs_rushing[['player_display_name', 'player_gsis_id']], 
                                                        on = 'player_display_name', how = 'inner').drop_duplicates()

In [87]:
ngs_avg_stats_rb

,season_19,player_display_name,efficiency_19,percent_attempts_gte_eight_defenders_19,avg_time_to_los_19,rush_attempts_19,rush_yards_19,expected_rush_yards_19,rush_yards_over_expected_19,avg_rush_yards_19,rush_yards_over_expected_per_att_19,rush_pct_over_expected_19,rush_touchdowns_19,season_20,efficiency_20,percent_attempts_gte_eight_defenders_20,avg_time_to_los_20,rush_attempts_20,rush_yards_20,expected_rush_yards_20,rush_yards_over_expected_20,avg_rush_yards_20,rush_yards_over_expected_per_att_20,rush_pct_over_expected_20,rush_touchdowns_20,season,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,expected_rush_yards,rush_yards_over_expected,avg_rush_yards,rush_yards_over_expected_per_att,rush_pct_over_expected,rush_touchdowns,season_22,efficiency_22,percent_attempts_gte_eight_defenders_22,avg_time_to_los_22,rush_attempts_22,rush_yards_22,expected_rush_yards_22,rush_yards_over_expected_22,avg_rush_yards_22,rush_yards_over_expected_per_att_22,rush_pct_over_expected_22,rush_touchdowns_22,player_gsis_id
0,2019.0,Aaron Jones,5.027324,20.654669,2.756946,29.000000,133.625000,120.666095,10.583905,4.380245,0.218970,0.387361,2.000000,2020.0,4.084915,19.359120,2.724828,26.800,147.200000,128.828728,17.037939,5.335387,0.582763,0.397022,1.200000,2021.0,4.079515,14.666474,2.689007,28.272727,130.363636,122.644690,7.355310,4.540392,0.237629,0.395922,0.636364,2022.0,3.992047,7.459673,2.852449,31.666667,172.916667,147.219792,16.530208,5.485285,0.547248,0.423547,0.333333,00-0033293
54,2019.0,Adrian Peterson,4.276318,30.083278,2.854776,29.000000,125.214286,109.030173,15.755541,4.259579,0.493684,0.412969,0.714286,2020.0,4.291046,30.683327,2.616606,32.500,123.625000,139.261264,-18.136264,3.716083,-0.610254,0.292767,1.625000,2021.0,8.602485,38.636364,3.026863,10.500000,18.500000,30.242605,-11.742605,1.777273,-1.114331,0.336364,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00-0025394
78,2019.0,Alexander Mattison,4.165529,39.126007,2.658618,30.600000,138.000000,126.483395,9.316605,4.386271,0.260979,0.414803,0.400000,2020.0,3.886056,37.250000,2.788746,31.800,147.200000,123.722453,22.277547,4.598929,0.808200,0.472957,0.600000,2021.0,4.978309,27.633302,2.752494,38.333333,144.500000,162.004132,-17.170799,3.622771,-0.692936,0.328137,0.833333,2022.0,2.928519,40.000000,2.457667,10.000000,54.000000,49.042091,4.957909,5.400000,0.495791,0.500000,2.000000,00-0034972
95,2019.0,Alvin Kamara,4.120010,16.557355,2.755820,25.384615,117.769231,111.459425,4.155959,4.675559,0.228808,0.355430,0.615385,2020.0,4.238100,9.622421,2.742201,27.000,134.692308,121.882365,10.656096,4.854057,0.190560,0.377381,2.230769,2021.0,4.893968,34.894280,2.788500,36.307692,137.769231,156.208035,-20.438804,3.666814,-0.748897,0.292114,0.615385,2022.0,4.009596,23.655212,2.766934,34.416667,140.500000,143.080144,-2.580144,4.057628,-0.045242,0.406821,0.333333,00-0033906
146,2019.0,Austin Ekeler,4.123590,14.343434,3.022346,38.200000,162.200000,169.471640,-16.471640,4.420410,-0.066063,0.374761,1.200000,2020.0,3.980014,10.194888,2.869136,26.875,124.250000,118.829049,5.170951,4.655512,0.376423,0.395029,0.250000,2021.0,3.791187,22.800214,2.680179,26.466667,117.400000,118.535670,-1.802337,4.496553,-0.060587,0.386903,1.600000,2022.0,4.218072,17.717010,2.739774,27.428571,125.000000,109.374603,13.482540,4.708127,0.612292,0.400073,1.785714,00-0033699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2283,NaN,Rachaad White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022.0,4.146888,22.889320,2.817926,33.333333,127.666667,130.681590,-3.014923,3.805846,-0.124826,0.358546,0.166667,00-0037256
2289,NaN,Travis Etienne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [88]:
ngs_avg_collective_rb

,season,player_display_name,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,expected_rush_yards,rush_yards_over_expected,avg_rush_yards,rush_yards_over_expected_per_att,rush_pct_over_expected,rush_touchdowns,player_gsis_id
0,2019,Aaron Jones,5.027324,20.654669,2.756946,29.000000,133.625000,120.666095,10.583905,4.380245,0.218970,0.387361,2.000000,00-0033293
54,2020,Aaron Jones,4.084915,19.359120,2.724828,26.800000,147.200000,128.828728,17.037939,5.335387,0.582763,0.397022,1.200000,00-0033293
108,2021,Aaron Jones,4.079515,14.666474,2.689007,28.272727,130.363636,122.644690,7.355310,4.540392,0.237629,0.395922,0.636364,00-0033293
162,2022,Aaron Jones,3.992047,7.459673,2.852449,31.666667,172.916667,147.219792,16.530208,5.485285,0.547248,0.423547,0.333333,00-0033293
216,2019,Adrian Peterson,4.276318,30.083278,2.854776,29.000000,125.214286,109.030173,15.755541,4.259579,0.493684,0.412969,0.714286,00-0025394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7291,2022,Rachaad White,4.146888,22.889320,2.817926,33.333333,127.666667,130.681590,-3.014923,3.805846,-0.124826,0.358546,0.166667,00-0037256
7297,2022,Travis Etienne,3.773225,27.371651,2.714666,33.416667,168.583333,135.194666,31.555334,5.177648,0.976272,0.470161,0.750000,00-0036973
7309,2022,Tyler Allgeier,4.069969,38.988832,2.860562,28.428571,140.714286,116.003222,17.853921,5.083750,0.622720,0.430524,0.428571,00-0037263
7323,2022,Tyrion Davis-Price,5.813030,42.857143,2.642100,14.000000,33.000000,36.527750,-3.527750,2.357143,-0.251982,0.357143,0.000000,00-0037827


### Quarterbacks (passing)

In [89]:
ngs_passing.sample(5)

,season,season_type,week,player_display_name,player_position,team_abbr,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,attempts,pass_yards,pass_touchdowns,interceptions,passer_rating,completions,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id,player_first_name,player_last_name,player_jersey_number,player_short_name
2764,2020,REG,15,Drew Brees,QB,NO,2.718441,6.767333,7.220909,-0.453576,29.411765,50.803961,-0.688182,34,234,3,1,84.681373,15,44.117647,59.225933,-15.108286,21.315339,50.803961,00-0020531,Drew,Brees,9,D.Brees
3321,2021,REG,14,Ryan Tannehill,QB,TEN,2.582452,3.420000,4.456667,-1.036667,3.225806,27.387444,-4.510000,31,191,0,0,81.518817,20,64.516129,73.813213,-9.297084,17.115732,33.871529,00-0029701,Ryan,Tannehill,17,R.Tannehill
2780,2020,REG,15,Russell Wilson,QB,SEA,2.708185,3.353889,5.030769,-1.676880,18.518519,23.637546,-4.007692,27,121,1,1,73.225309,18,66.666667,65.357796,1.308871,17.938653,43.296606,00-0029263,Russell,Wilson,3,R.Wilson
3300,2021,REG,13,Dak Prescott,QB,DAL,2.875900,5.751154,8.997250,-3.246096,30.000000,30.869605,0.447250,40,238,1,1,78.958333,26,65.000000,61.480474,3.519526,22.791463,48.332384,00-0033077,Rayne,Prescott,4,D.Prescott
3581,2022,REG,2,Davis Mills,QB,HOU,2.566737,4.617368,8.530270,-3.912902,18.421053,26.773509,-1.253514,38,177,0,0,63.157895,19,50.000000,62.778657,-12.778657,22.664155,45.873533,00-0036898,Davis,Mills,10,D.Mills


In [90]:
# like RBs, we only want regular season

ngs_passing = ngs_passing.loc[ngs_passing['season_type'] == 'REG']

# jersey number, first last name, and short name don't matter
to_drop = ['week', 'season_type', 'player_position', 'player_first_name', 'player_last_name', 
           'player_jersey_number', 'player_short_name']

ngs_passing = ngs_passing.drop(columns = to_drop)

# getting average stats by player by year
ngs_avg_stats_qb_collective = ngs_passing.groupby(['season','player_display_name', 'player_gsis_id'], as_index = False).mean(numeric_only = True)

# breaking into their own tables to then merge side by side with each year
ngs_avg_stats_19 = ngs_avg_stats_qb_collective[ngs_avg_stats_qb_collective.season == 2019]
ngs_avg_stats_20 = ngs_avg_stats_qb_collective[ngs_avg_stats_qb_collective.season == 2020]
ngs_avg_stats_21 = ngs_avg_stats_qb_collective[ngs_avg_stats_qb_collective.season == 2021]
ngs_avg_stats_22 = ngs_avg_stats_qb_collective[ngs_avg_stats_qb_collective.season == 2022]


# now to merge
# larger table with columns for each year's stats
ngs_avg_stats_qb = ngs_avg_stats_19.merge(ngs_avg_stats_20, on = 'player_gsis_id', how = 'outer', suffixes = ('_19', '_20'))
ngs_avg_stats_qb = ngs_avg_stats_qb.merge(ngs_avg_stats_21, on = 'player_gsis_id', how = 'outer', suffixes = ('', '_21')) \
                    .merge(ngs_avg_stats_22, on = 'player_gsis_id', how = 'outer', suffixes = ('', '_22'))



In [91]:
# bringing player id back in

ngs_avg_collective_qb = ngs_avg_stats_qb_collective.merge(ngs_passing[['player_display_name', 'player_gsis_id']], 
                                     
                                          on = 'player_display_name', how = 'inner').drop_duplicates()


ngs_avg_stats_qb = ngs_avg_stats_qb.merge(ngs_passing[['player_display_name', 'player_gsis_id']], 
                                                        on = 'player_display_name', how = 'inner').drop_duplicates()

### Wide Receivers

In [92]:
ngs_receiving.sample(5)

,season,season_type,week,player_display_name,player_position,team_abbr,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,receptions,targets,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_jersey_number,player_short_name
4992,2019,REG,6,Robby Anderson,WR,NYJ,5.307143,1.994765,15.148750,47.029376,5,8,62.500000,125.0,1,12.644000,9.363568,3.280432,00-0032688,Robby,Anderson,11,Ro.Anderson
9898,2022,REG,12,Van Jefferson,WR,LAR,4.866667,3.409973,11.771667,45.007328,3,6,50.000000,29.0,1,-0.666667,0.470358,-1.137025,00-0036415,Vanchii,Jefferson,12,V.Jefferson
7020,2020,REG,14,Nelson Agholor,WR,LV,6.075000,2.937447,14.271111,41.425577,5,9,55.555556,100.0,1,4.350000,4.978350,-0.628350,00-0031549,Nelson,Agholor,15,N.Agholor
4501,2019,REG,0,Zach Pascal,WR,IND,5.731806,2.824933,11.546667,21.117230,41,72,56.944444,607.0,5,5.864634,5.309553,0.555081,00-0033251,Zachary,Pascal,14,Z.Pascal
9972,2022,REG,13,Amari Cooper,WR,CLE,4.633333,3.762231,8.986667,57.970183,4,9,44.444444,40.0,0,4.070000,3.573462,0.496538,00-0031544,Amari,Cooper,2,A.Cooper


In [93]:
# like RBs, we only want regular season

ngs_receiving = ngs_receiving.loc[ngs_receiving['season_type'] == 'REG']

# but, we only want WR for position here, not TE

ngs_wr = ngs_receiving.loc[ngs_receiving['player_position'] == 'WR']

# jersey number, first last name, and short name don't matter
to_drop = ['week', 'season_type', 'player_position', 'player_first_name', 'player_last_name', 
           'player_jersey_number', 'player_short_name']

ngs_wr = ngs_wr.drop(columns = to_drop)

# getting average stats by player by year
ngs_avg_stats_wr_collective = ngs_wr.groupby(['season','player_display_name'], as_index = False).mean(numeric_only = True)

# breaking into their own tables to then merge side by side with each year
ngs_avg_stats_19 = ngs_avg_stats_wr_collective[ngs_avg_stats_wr_collective.season == 2019]
ngs_avg_stats_20 = ngs_avg_stats_wr_collective[ngs_avg_stats_wr_collective.season == 2020]
ngs_avg_stats_21 = ngs_avg_stats_wr_collective[ngs_avg_stats_wr_collective.season == 2021]
ngs_avg_stats_22 = ngs_avg_stats_wr_collective[ngs_avg_stats_wr_collective.season == 2022]


# now to merge
ngs_avg_stats_wr = ngs_avg_stats_19.merge(ngs_avg_stats_20, on = 'player_display_name', how = 'outer', suffixes = ('_19', '_20'))
ngs_avg_stats_wr = ngs_avg_stats_wr.merge(ngs_avg_stats_21, on = 'player_display_name', how = 'outer', suffixes = ('', '_21')) \
                    .merge(ngs_avg_stats_22, on = 'player_display_name', how = 'outer', suffixes = ('', '_22'))

In [94]:
# bringing player id back in

ngs_avg_collective_wr = ngs_avg_stats_wr_collective.merge(ngs_receiving[['player_display_name', 'player_gsis_id']], 
                                                        on = 'player_display_name', how = 'inner').drop_duplicates()

ngs_avg_stats_wr = ngs_avg_stats_wr.merge(ngs_wr[['player_display_name', 'player_gsis_id']], 
                                                        on = 'player_display_name', how = 'inner').drop_duplicates()

In [95]:
ngs_avg_stats_wr.head()

,season_19,player_display_name,avg_cushion_19,avg_separation_19,avg_intended_air_yards_19,percent_share_of_intended_air_yards_19,receptions_19,targets_19,catch_percentage_19,yards_19,rec_touchdowns_19,avg_yac_19,avg_expected_yac_19,avg_yac_above_expectation_19,season_20,avg_cushion_20,avg_separation_20,avg_intended_air_yards_20,percent_share_of_intended_air_yards_20,receptions_20,targets_20,catch_percentage_20,yards_20,rec_touchdowns_20,avg_yac_20,avg_expected_yac_20,avg_yac_above_expectation_20,season,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,receptions,targets,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,season_22,avg_cushion_22,avg_separation_22,avg_intended_air_yards_22,percent_share_of_intended_air_yards_22,receptions_22,targets_22,catch_percentage_22,yards_22,rec_touchdowns_22,avg_yac_22,avg_expected_yac_22,avg_yac_above_expectation_22,player_gsis_id
0,2019.0,A.J. Brown,4.998002,2.325755,12.586335,34.786790,9.666667,15.777778,59.668295,194.555556,1.444444,8.392478,4.037331,4.355146,2020.0,5.459333,2.474082,11.095020,36.469486,9.928571,14.857143,66.950284,152.071429,1.571429,6.530935,3.642770,2.888166,2021.0,6.147317,2.680218,12.029758,46.975805,10.416667,17.000000,60.611322,143.500000,0.833333,3.787133,3.663129,0.124003,2022.0,5.641254,2.730035,11.734585,42.014711,10.294118,16.823529,62.001262,175.588235,1.294118,6.811628,4.558055,2.253573,00-0035676
52,2019.0,Adam Humphries,5.925798,2.782947,7.091745,15.480636,11.200000,14.800000,72.411348,113.600000,0.400000,3.276297,3.869869,-0.593572,2020.0,6.292988,2.982571,8.382262,20.966688,5.250000,6.500000,81.547619,50.000000,0.500000,2.221917,2.953779,-0.731862,2021.0,6.306215,3.172473,6.206948,12.752039,9.142857,14.142857,62.015032,85.571429,0.000000,4.168160,4.775217,-0.607057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00-0032009
68,2019.0,Adam Thielen,6.427153,2.900382,12.714965,38.849689,8.833333,13.833333,63.472222,123.500000,1.666667,4.025802,4.177237,-0.151436,2020.0,5.949418,2.580153,13.131793,38.072344,10.769231,15.769231,66.287352,136.384615,2.076923,3.331271,2.841690,0.489581,2021.0,5.911510,3.163530,9.566764,31.250087,11.636364,16.545455,68.465263,124.727273,1.818182,3.640224,3.466008,0.174215,2022.0,6.134638,2.725963,10.038573,27.393536,9.923077,15.076923,63.417577,100.461538,0.769231,2.359794,3.450323,-1.090528,00-0030035
111,2019.0,Albert Wilson,7.035792,3.490992,4.896085,9.164434,9.875000,13.625000,76.675307,81.750000,0.125000,4.166960,4.095299,0.071661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0,6.722083,3.737623,3.132812,6.062925,4.250000,6.250000,70.000000,37.500000,0.000000,7.047875,6.933007,0.114868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00-0030669
123,2019.0,Alex Erickson,6.613852,2.550214,9.160670,21.850441,8.444444,15.444444,51.971917,106.555556,0.000000,6.163606,4.362931,1.800674,2020.0,5.635000,3.749726,8.580000,15.986088,6.000000,6.000000,100.000000,88.000000,0.000000,6.086667,6.460935,-0.374268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00-0032543


### Tight Ends

In [96]:
# now we want TEs

ngs_te = ngs_receiving.loc[ngs_receiving['player_position'] == 'TE']

# jersey number, first last name, and short name don't matter
to_drop = ['week', 'season_type', 'player_position', 'player_first_name', 'player_last_name', 
           'player_jersey_number', 'player_short_name']

ngs_te = ngs_te.drop(columns = to_drop)

# getting average stats by player by year
# saving as collective
ngs_avg_stats_te_collective = ngs_te.groupby(['season','player_display_name'], as_index = False).mean(numeric_only = True)

# breaking into their own tables to then merge side by side with each year
ngs_avg_stats_19 = ngs_avg_stats_te_collective[ngs_avg_stats_te_collective.season == 2019]
ngs_avg_stats_20 = ngs_avg_stats_te_collective[ngs_avg_stats_te_collective.season == 2020]
ngs_avg_stats_21 = ngs_avg_stats_te_collective[ngs_avg_stats_te_collective.season == 2021]
ngs_avg_stats_22 = ngs_avg_stats_te_collective[ngs_avg_stats_te_collective.season == 2022]


# now to merge
ngs_avg_stats_te = ngs_avg_stats_19.merge(ngs_avg_stats_20, on = 'player_display_name', how = 'outer', suffixes = ('_19', '_20'))
ngs_avg_stats_te = ngs_avg_stats_te.merge(ngs_avg_stats_21, on = 'player_display_name', how = 'outer', suffixes = ('', '_21')) \
                    .merge(ngs_avg_stats_22, on = 'player_display_name', how = 'outer', suffixes = ('', '_22'))


In [97]:
# bringing player id back in

ngs_avg_collective_te = ngs_avg_stats_te_collective.merge(ngs_receiving[['player_display_name', 'player_gsis_id']], 
                                                        on = 'player_display_name', how = 'inner').drop_duplicates()

ngs_avg_stats_te = ngs_avg_stats_te.merge(ngs_wr[['player_display_name', 'player_gsis_id']], 
                                                        on = 'player_display_name', how = 'inner').drop_duplicates()

In [98]:
ngs_all_wide = pd.concat([ngs_avg_stats_qb, ngs_avg_stats_rb,
                              ngs_avg_stats_wr, ngs_avg_stats_te])

ngs_all_tall = pd.concat([ngs_avg_collective_qb, ngs_avg_collective_rb,
                         ngs_avg_collective_wr, ngs_avg_collective_te])

ngs_all_tall.head()

,season,player_display_name,player_gsis_id_x,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,attempts,pass_yards,pass_touchdowns,interceptions,passer_rating,completions,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id_y,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,expected_rush_yards,rush_yards_over_expected,avg_rush_yards,rush_yards_over_expected_per_att,rush_pct_over_expected,rush_touchdowns,player_gsis_id,avg_cushion,avg_separation,percent_share_of_intended_air_yards,receptions,targets,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation
0,2019,AJ McCarron,00-0031288,3.023472,4.192857,8.103750,-3.910893,16.666667,34.452502,0.572500,36.000000,225.000000,0.000000,1.000000,65.162037,21.000000,58.333333,59.119197,-0.785863,20.992778,46.981911,00-0031288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,Aaron Rodgers,00-0023459,2.895882,5.423481,8.817561,-3.394081,13.803295,41.824807,-0.023591,66.941176,470.823529,3.058824,0.470588,95.671160,41.529412,62.443619,64.581977,-2.138358,23.081483,53.298956,00-0023459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,2020,Aaron Rodgers,00-0023459,2.726101,5.402501,7.986633,-2.584132,10.579508,45.733018,-0.501080,61.882353,505.764706,5.647059,0.588235,119.778930,43.764706,71.132776,67.953422,3.179354,22.344334,53.654674,00-0023459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,2021,Aaron Rodgers,00-0023459,2.629163,4.861571,7.641316,-2.779745,16.516486,43.783561,-0.641163,62.470588,484.117647,4.352941,0.470588,112.464614,43.058824,69.430188,66.092304,3.337885,21.579879,52.007562,00-0023459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,2022,Aaron Rodgers,00-0023459,2.701436,5.128770,8.027893,-2.899122,15.095707,40.490173,-0.504206,60.222222,410.555556,2.888889,1.333333,93.531095,38.888889,65.154708,66.350627,-1.195919,21.550354,52.482642,00-0023459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Merging all data

Goal is to create overall collective DF with data from each season, with season as its own columns

## DFs we have:
    
    ff_all_collective: all fantasy points and scoring from each player from 2019-2022
    ngs_all_collective: all ngs stats from each player from 2019-2022
    qbr_collective: all qbr stats for each qb from 2019-2022
    team_all_collective: all team stats from 2019-2022
    
We'll need to join these on each other to get stats from each player for each season

In [99]:
# creating overall collective DF with data from each season, with season as its own column

qbr_collective.head()

,season,team_abb,player_id,qbr_total,pts_added,qb_plays,epa_total,pass,run,exp_sack,qbr_raw,sack,name_display
937,2019,BAL,3916387,83.0,66.7,613,103.7,55.0,39.1,0,82.3,-7.4,Lamar Jackson
938,2019,KC,3139477,77.7,55.8,585,97.3,71.6,14.3,0,79.4,-6.5,Patrick Mahomes
939,2019,NO,2580,73.3,33.7,419,62.6,53.1,1.6,0,75.7,-6.0,Drew Brees
940,2019,DAL,2577417,71.9,48.1,690,93.1,70.7,10.0,0,72.8,-9.7,Dak Prescott
941,2019,SEA,14881,71.5,43.0,674,90.9,58.3,10.6,0,71.2,-20.6,Russell Wilson


In [100]:
# since qbr_Collective doesn't have the same id, we will join on player name
# need to ensure it undergoes the same type of cleaning

text_to_remove = ['+', '*', ' Jr.',' III', ' II', ' Sr.']

# to ensure we escape special characters so there are no errors
text_to_remove_escaped = [re.escape(text) for text in text_to_remove]

qbr_collective['name_display'] = qbr_collective['name_display'].str.replace('|'.join(text_to_remove_escaped), '',regex= True )
ngs_all_tall['player_display_name'] = ngs_all_tall['player_display_name'].str.replace('|'.join(text_to_remove_escaped), '',regex= True )
injuries_by_season['full_name'] = injuries_by_season['full_name'].str.replace('|'.join(text_to_remove_escaped), '',regex= True )

qbr_collective.head()

,season,team_abb,player_id,qbr_total,pts_added,qb_plays,epa_total,pass,run,exp_sack,qbr_raw,sack,name_display
937,2019,BAL,3916387,83.0,66.7,613,103.7,55.0,39.1,0,82.3,-7.4,Lamar Jackson
938,2019,KC,3139477,77.7,55.8,585,97.3,71.6,14.3,0,79.4,-6.5,Patrick Mahomes
939,2019,NO,2580,73.3,33.7,419,62.6,53.1,1.6,0,75.7,-6.0,Drew Brees
940,2019,DAL,2577417,71.9,48.1,690,93.1,70.7,10.0,0,72.8,-9.7,Dak Prescott
941,2019,SEA,14881,71.5,43.0,674,90.9,58.3,10.6,0,71.2,-20.6,Russell Wilson


In [101]:
# season to int
convert = {'season': int}

team_all_collective = team_all_collective.astype(convert)


ff_all_collective = ff_all_collective.astype({'season': int})


In [102]:
ff_all_collective = ff_all_collective.drop_duplicates(subset = ['player_id', 'season'])

In [103]:
ngs_all_tall.head()

,season,player_display_name,player_gsis_id_x,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,attempts,pass_yards,pass_touchdowns,interceptions,passer_rating,completions,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id_y,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,expected_rush_yards,rush_yards_over_expected,avg_rush_yards,rush_yards_over_expected_per_att,rush_pct_over_expected,rush_touchdowns,player_gsis_id,avg_cushion,avg_separation,percent_share_of_intended_air_yards,receptions,targets,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation
0,2019,AJ McCarron,00-0031288,3.023472,4.192857,8.103750,-3.910893,16.666667,34.452502,0.572500,36.000000,225.000000,0.000000,1.000000,65.162037,21.000000,58.333333,59.119197,-0.785863,20.992778,46.981911,00-0031288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,Aaron Rodgers,00-0023459,2.895882,5.423481,8.817561,-3.394081,13.803295,41.824807,-0.023591,66.941176,470.823529,3.058824,0.470588,95.671160,41.529412,62.443619,64.581977,-2.138358,23.081483,53.298956,00-0023459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,2020,Aaron Rodgers,00-0023459,2.726101,5.402501,7.986633,-2.584132,10.579508,45.733018,-0.501080,61.882353,505.764706,5.647059,0.588235,119.778930,43.764706,71.132776,67.953422,3.179354,22.344334,53.654674,00-0023459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,2021,Aaron Rodgers,00-0023459,2.629163,4.861571,7.641316,-2.779745,16.516486,43.783561,-0.641163,62.470588,484.117647,4.352941,0.470588,112.464614,43.058824,69.430188,66.092304,3.337885,21.579879,52.007562,00-0023459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,2022,Aaron Rodgers,00-0023459,2.701436,5.128770,8.027893,-2.899122,15.095707,40.490173,-0.504206,60.222222,410.555556,2.888889,1.333333,93.531095,38.888889,65.154708,66.350627,-1.195919,21.550354,52.482642,00-0023459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Merging

In [104]:
all_collective = ff_all_collective.merge(ngs_all_tall, left_on = ['player_name', 'season'],
                                      right_on = ['player_display_name', 'season'])\
                     .merge(qbr_collective,how = 'left',left_on = ['player_name', 'season'],
                                        right_on = ['name_display', 'season']).merge(\
                                        team_all_collective, left_on = ['team', 'season'], 
                                       right_on = ['Abbreviation', 'season'], how = 'left',
                                      suffixes = ('', '_tm')) \
                    .merge(injuries_by_season, how = 'left', left_on = ['player_id_x','season'],
                                        right_on = ['gsis_id', 'season'])

In [105]:
all_collective.loc[all_collective['player_name'] == 'Tom Brady']

,team,player_name,position,player_id_x,height,weight,years_exp,tm,pos,age,g,gs,tgt,rec,passingyds,passingtd,passingatt,rushingyds,rushingtd,rushingatt,receivingyds,receivingtd,fantasypoints,int,fumbles,fumbleslost,adp,name,season,cmp,y/a,y/r,totaltds_22,2pm,2pp,ppr,dkpt,fdpt,vbd,posrank,player_display_name,player_gsis_id_x,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,attempts,pass_yards,pass_touchdowns,interceptions,passer_rating,completions,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id_y,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,expected_rush_yards,rush_yards_over_expected,avg_rush_yards,rush_yards_over_expected_per_att,rush_pct_over_expected,rush_touchdowns,player_gsis_id,avg_cushion,avg_separation,percent_share_of_intended_air_yards,receptions,targets,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,team_abb,player_id_y,qbr_total,pts_added,qb_plays,epa_total,pass,run,exp_sack,qbr_raw,sack,name_display,Team,Pts_scored,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Pts_allowed,Yds_alowed,PassYds_allowed,Rushyds_allowed,Team_takeaways,Sp. Tms,losses,ties,wins,team_wins,Team_games,Abbreviation,gsis_id,full_name,games_out,games_questionable,games_out_or_questionable
226,NE,Tom Brady,QB,00-0019596,76.0,225.0,19.0,NWE,QB,42.0,16.0,16.0,0.0,0.0,4057.0,24.0,613.0,34.0,3.0,26.0,0.0,0.0,261.68,8.0,4.0,1.0,115,Tom Brady,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tom Brady,00-0019596,2.733361,5.625395,7.596742,-1.971347,15.233427,39.629354,-1.567239,72.117647,477.294118,2.823529,0.941176,89.035389,43.882353,60.852555,63.830573,-2.978018,20.504887,45.989556,00-0019596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NE,2330.0,55.7,8.9,691.0,61.0,40.0,4.5,0.0,54.5,-11.0,Tom Brady,New England Patriots,420.0,338.0,5664.0,3961.0,1703.0,15.0,35.46,169.58,225.0,4414.0,2886.0,1528.0,36.0,-16.48,4.0,0.0,12.0,12.0,NaN,NE,00-0019596,Tom Brady,0.0,1.0,1.0
695,TB,Tom Brady,QB,00-0019596,76.0,225.0,20.0,TAM,QB,43.0,16.0,16.0,0.0,0.0,4633.0,40.0,610.0,6.0,3.0,30.0,0.0,0.0,338.0,12.0,4.0,1.0,104,Tom Brady,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tom Brady,00-0019596,2.574172,7.065031,9.384336,-2.319304,14.558931,43.783447,0.689700,71.764706,545.058824,4.705882,1.411765,102.871340,47.176471,66.019528,64.165692,1.853836,22.189925,52.793336,00-0019596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TB,2330.0,72.5,49.2,681.0,94.7,90.4,-3.1,0.0,73.5,-8.6,Tom Brady,Tampa Bay Buccaneers,492.0,364.0,6145.0,4626.0,1519.0,17.0,200.73,-23.90,355.0,5234.0,3945.0,1289.0,25.0,-33.68,5.0,0.0,11.0,11.0,NaN,TB,00-0019596,Tom Brady,0.0,0.0,0.0
1070,TB,Tom Brady,QB,00-0019596,76.0,225.0,21.0,TAM,QB,44.0,17.0,17.0,0.0,0.0,5316.0,43.0,719.0,81.0,2.0,28.0,0.0,0.0,375.0,12.0,4.0,3.0,77,Tom Brady,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tom Brady,00-0019596,2.489198,5.617218,7.969081,-2.351864,13.663329,41.327304,-0.366083,79.888889,590.555556,4.777778,1.333333,101.910597,53.888889,67.352689,68.806056,-1.453367,20.706071,51.028989,00-0019596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TB,2330.0,68.1,40.1,798.0,118.5,109.8,4.9,0.0,67.1,-8.7,Tom Brady,Tampa Bay Buccaneers,511.0,404.0,6901.0,5229.0,1672.0,19.0,214.94,-14.28,353.0,5635.0,4062.0,1573.0,29.0,-38.96,4.0,0.0,13.0,13.0,NaN,TB,00-0019596,Tom Brady,0.0,0.0,0.0
1424,TB,Tom Brady,QB,00-0019596,76.0,225.0,22.0,TAM,QB,45,17,17,1,0,4694,25,733,-1,1,29,0,0,272,9,5,4,81,Tom Brady,2022,490,-0.03,,1,,2,271.7,287.7,280.7,,12,Tom Brady,00-0019596,2.442485,4.691857,7.096871,-2.405013,14.948969,40.278566,-1.352540,81.444444,521.555556,2.777778,1.000000,91.931012,54.444444,67.233502,68.017720,-

# Creating data from previous seasons

In [107]:
# what if i can join the columns i need from here onto all_collective...

team_all_collective['next_season'] = team_all_collective['season'] + 1

team_nxt_season = team_all_collective[['PassY', 'RushY', 'TotYd', 'Pts_scored', 
                                       'team_wins', 'next_season', 'Team']]

In [108]:
team_nxt_season.head()

,PassY,RushY,TotYd,Pts_scored,team_wins,next_season,Team
0,3477.0,1990.0,5467.0,361.0,5.5,2020,Arizona Cardinals
1,3916.0,2237.0,6153.0,410.0,8.0,2021,Arizona Cardinals
2,4276.0,2076.0,6352.0,449.0,11.0,2022,Arizona Cardinals
3,3626,1873,5499,340,NaN,2023,Arizona Cardinals
4,4714.0,1361.0,6075.0,381.0,7.0,2020,Atlanta Falcons


Need to bring in 2022 for 2023 season as a predictor. Need player's teams in 2023 for that. Let's get that from the rank 23 data

In [109]:
rosters23_rel.head()

,team,player_name,position,player_id,height,weight,years_exp
60,ARI,Jeff Driskel,QB,00-0032436,76.0,235.0,7
61,ARI,Kyler Murray,QB,00-0035228,70.0,207.0,4
62,ARI,Clayton Tune,QB,00-0038582,75.0,220.0,0
63,ARI,Colt McCoy,QB,00-0027688,73.0,212.0,13
64,ARI,Joshua Dobbs,QB,00-0033949,75.0,216.0,6


## Adding 2023 Roster Data

In [110]:
rosters23_rel['season'] = 2023

rosters23_rel_ = rosters23_rel.merge(team_all_collective[['Team', 'Abbreviation']], how = 'right', left_on = 'team', right_on = 'Abbreviation')
rosters23_rel_.rename(columns = {'Abbreviation': 'tm', 'Pos': 'position'}, inplace = True)

rosters23_rel_.drop(columns = ['team'], inplace = True)
rosters23_rel_.drop_duplicates(subset = 'player_name', keep = 'first', inplace = True)



C:\Users\steph\AppData\Local\Temp/ipykernel_16852/4152542524.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rosters23_rel['season'] = 2023


In [111]:
all_collective_23 = pd.concat([all_collective, rosters23_rel_[['player_name', 'Team', 'position', 'season', 'tm']]], axis = 0 )

In [112]:
# bringing the 'next season' team data in

all_collective_23 = all_collective_23.merge(team_nxt_season, how = 'left',
                                        left_on = ['Team', 'season'], right_on = ['Team', 'next_season'],
                                       suffixes = ['', '_last_yr_tm_stat'])

In [113]:
all_collective_23.sample(6)

,team,player_name,position,player_id_x,height,weight,years_exp,tm,pos,age,g,gs,tgt,rec,passingyds,passingtd,passingatt,rushingyds,rushingtd,rushingatt,receivingyds,receivingtd,fantasypoints,int,fumbles,fumbleslost,adp,name,season,cmp,y/a,y/r,totaltds_22,2pm,2pp,ppr,dkpt,fdpt,vbd,posrank,player_display_name,player_gsis_id_x,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,attempts,pass_yards,pass_touchdowns,interceptions,passer_rating,completions,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id_y,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,expected_rush_yards,rush_yards_over_expected,avg_rush_yards,rush_yards_over_expected_per_att,rush_pct_over_expected,rush_touchdowns,player_gsis_id,avg_cushion,avg_separation,percent_share_of_intended_air_yards,receptions,targets,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,team_abb,player_id_y,qbr_total,pts_added,qb_plays,epa_total,pass,run,exp_sack,qbr_raw,sack,name_display,Team,Pts_scored,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Pts_allowed,Yds_alowed,PassYds_allowed,Rushyds_allowed,Team_takeaways,Sp. Tms,losses,ties,wins,team_wins,Team_games,Abbreviation,gsis_id,full_name,games_out,games_questionable,games_out_or_questionable,PassY_last_yr_tm_stat,RushY_last_yr_tm_stat,TotYd_last_yr_tm_stat,Pts_scored_last_yr_tm_stat,team_wins_last_yr_tm_stat,next_season
1946,NaN,Joshua Kelley,RB,NaN,NaN,NaN,NaN,LAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles Chargers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4584,1524,6108,391,NaN,2023.0
2177,NaN,Connor Heyward,TE,NaN,NaN,NaN,NaN,PIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pittsburgh Steelers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3411,2073,5484,308,NaN,2023.0
2089,NaN,Gary Brightwell,RB,NaN,NaN,NaN,NaN,NYG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New York Giants,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3157,2519,5676,365,NaN,2023.0
1490,NaN,Logan Woodside,QB,NaN,NaN,NaN,NaN,ATL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Atlanta Falcons,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2699,2718,5417,365,NaN,2023.0
504,IND,Nyheim Hines,RB,00-0034367,69.0,198.0,2.0,IND,RB,24.0,16.0,2.0,76.0,63.0,0.0,0.0,0.0,380.0,3.0,89.0,482.0,4.0,130.0,0.0,0.0,0.0,156,Nyheim Hines,2020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [114]:
all_collective_23.sort_values(by = ['player_name', 'season'], inplace = True)

all_collective_23['team_nxt_season'] = all_collective_23.groupby('player_name')['Team'].shift(-1)

qb_nxt_season = all_collective_23[['player_name', 'position', 'player_id_x', 'passingyds', 
                                'passingtd', 'avg_air_distance', 'pts_added', 
                                'season', 'next_season', 
                                'Team', 'team_nxt_season']].loc[all_collective_23['position'] == 'QB']
qb_nxt_season.head()

,player_name,position,player_id_x,passingyds,passingtd,avg_air_distance,pts_added,season,next_season,Team,team_nxt_season
124,AJ McCarron,QB,00-0031288,NaN,NaN,20.992778,NaN,2019.0,NaN,Houston Texans,NaN
115,Aaron Rodgers,QB,00-0023459,4002.0,26.0,23.081483,10.9,2019.0,NaN,Green Bay Packers,Green Bay Packers
482,Aaron Rodgers,QB,00-0023459,4299.0,48.0,22.344334,77.7,2020.0,2020.0,Green Bay Packers,Green Bay Packers
857,Aaron Rodgers,QB,00-0023459,4115.0,37.0,21.579879,32.8,2021.0,2021.0,Green Bay Packers,Green Bay Packers
1226,Aaron Rodgers,QB,00-0023459,3695,26,21.550354,-14.3,2022.0,2022.0,Green Bay Packers,New York Jets


Was going to add rank data, but since 2023 can be changing, it may be better to ingest that in the modeling notebook, or at least, the notebook before it.

In [115]:
all_collective_23.shape

(2365, 129)

# Saving Data 

In [116]:
## Saving the two all encompassing DFs 
# (all, and all_collective -- which are players stats per player over all season, and player stats per season, respectively)
# going to feature engineer and visualize in another notebook
all_collective_23.to_csv("C:/Users/steph/OneDrive/Documents/Data Projects/Fantasy/ff_all_collective_seasons.csv")

In [117]:
# saving 2023 rank data to utilize in making current year prediction

ranks_23.to_csv("C:/Users/steph/OneDrive/Documents/Data Projects/Fantasy/ff_23_ranks_adp.csv")
rosters23_rel_.to_csv("C:/Users/steph/OneDrive/Documents/Data Projects/Fantasy/rosters_23.csv")